In [92]:
import skorch
from scipy.ndimage import gaussian_filter
from datajoint_tables import *
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')
from plotting import *
from sklearn.linear_model import Ridge
from Trials import *
from RNN import *
import pickle
from Perturbation import Perturbation
from LatentCircuitAnalysis import *
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from matplotlib.ticker import FormatStrFormatter
from scipy.optimize import curve_fit
%matplotlib tk


# Models

In [2]:
cond_frame = pd.DataFrame(data={'patience': [50]})
df = pd.DataFrame((Model()  & cond_frame  & 'r2>.9').proj('model_id','activation','lr','batch_size','patience','threshold','r2','epochs','lambda_r','lambda_o','l2_rate','l2_ortho').fetch())
df

,model_id,activation,lr,batch_size,patience,threshold,lambda_r,lambda_o,r2,epochs,l2_ortho,l2_rate
0,0HQrojHE,ReLU(),0.010000,128,50,0.00010,0.000000,0.000000,0.972088,500,2.393450e+00,0.015559
1,0KEHeLk3,ReLU(),0.010000,128,50,0.00010,0.000000,0.000000,0.979679,421,2.491250e+00,0.013572
2,0tCcR09J,ReLU(),0.010000,128,50,0.00010,1.000000,1.000000,0.964976,182,4.506420e-07,0.003587
3,1CE85dgv,ReLU(),0.010000,128,50,0.00010,0.000000,0.000000,0.958882,353,2.590170e+00,0.014448
4,1G9hHqfY,ReLU(),0.010000,128,50,0.00010,1.000000,1.000000,0.948257,222,5.340490e-06,0.002966
...,...,...,...,...,...,...,...,...,...,...,...,...
263,zHktk0xo,ReLU(),0.010000,128,50,0.00010,0.000000,1.000000,0.905384,284,7.415270e-05,0.015848
264,zn1rrskO,ReLU(),0.010000,128,50,0.00010,0.000000,1.000000,0.939016,371,8.080950e-05,0.018371
265,znh1iemU,ReLU(),0.010000,128,50,0.00010,0.000000,1.000000,0.960942,456,1.297220e-05,0.030832
266,ZnxvVg4S,ReLU(),0.010000,128,50,0.00010,1.000000,1.000000,0.973264,369,2.051490e-06,0.002940


In [154]:
pd.DataFrame((LCA() & 'model_id="PU3GbIco"').proj('valid_loss','r2_x').fetch()).sort_values('valid_loss')


,model_id,lca_id,r2_x,valid_loss
0,PU3GbIco,8WsHgYlC,0.0113,0.03415
4,PU3GbIco,OZRo7bFu,0.0113,0.03461
3,PU3GbIco,LHvSKA7U,0.0117,0.03561
1,PU3GbIco,cEhOl9Nt,0.0114,0.03778
2,PU3GbIco,HcV8H34n,0.0122,0.04131


In [138]:
Model() & 'model_id="7MGKnyFO"'

model_id unique model id,connectivity specify connectivity structure,n number of neurons,activation,lr learning rate,batch_size,patience,threshold,lambda_r firing rate regularization constant,lambda_o orthogonality regularization constant,lambda_w weight regularization,r2,valid_loss,train_loss,epochs,l2_ortho,l2_rate,l2_weight,l2_task,w_rec,w_in,w_out,embedding,train_loss_history,valid_loss_history
7MGKnyFO,large,150,ReLU(),0.010000,128,25,0.00010,0.010000,1.000000,0.000000,0.954489,0.00729629,0.00609157,128,7.56122e-08,0.0183412,0.0264064,0.0071128,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=


In [41]:
reduced_ids = LCA().fetch('model_id')
ids = (Model() & 'patience=25' & 'lambda_r=0' & 'lambda_o=1' & 'r2>.95').fetch('model_id')
not_reduced_models = pd.DataFrame(data={'model_id':[x for x in ids if x not in reduced_ids]})
not_reduced_models

,model_id
0,0DJahE7H
1,93ilB1QL
2,9t6LiUGL
3,a0ANAXM5
4,cYV1WMKX
5,dTiqVOoe
6,FBRQaFn4
7,iZFqxiQm
8,n7Rx5HAO
9,q5Wkjbo9


In [42]:
Model() & not_reduced_models

model_id unique model id,connectivity specify connectivity structure,n number of neurons,activation,lr learning rate,batch_size,patience,threshold,lambda_r firing rate regularization constant,lambda_o orthogonality regularization constant,lambda_w weight regularization,r2,valid_loss,train_loss,epochs,l2_ortho,l2_rate,l2_weight,l2_task,w_rec,w_in,w_out,embedding,train_loss_history,valid_loss_history
0DJahE7H,large,150,ReLU(),0.010000,128,25,0.00010,0.000000,1.000000,0.000000,0.959435,0.0063398,0.00505611,250,5.45339e-09,0.0438521,0.0289542,0.00633979,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
93ilB1QL,large,150,ReLU(),0.010000,128,25,0.00010,0.000000,1.000000,0.000000,0.963952,0.00563856,0.0119409,106,4.63427e-06,0.0238238,0.0259289,0.00563393,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
9t6LiUGL,large,150,ReLU(),0.010000,128,25,0.00010,0.000000,1.000000,0.000000,0.955617,0.00693959,0.00601391,152,3.00232e-06,0.0186886,0.0246198,0.00693659,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
a0ANAXM5,large,150,ReLU(),0.010000,128,25,0.00010,0.000000,1.000000,0.000000,0.964045,0.00562007,0.00465445,311,6.20096e-07,0.0179853,0.0335263,0.00561945,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
cYV1WMKX,large,150,ReLU(),0.010000,128,25,0.00010,0.000000,1.000000,0.000000,0.960878,0.00611582,0.0120554,190,1.51854e-06,0.0224454,0.0285867,0.0061143,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
dTiqVOoe,large,150,ReLU(),0.010000,128,25,0.00010,0.000000,1.000000,0.000000,0.96922,0.00483112,0.00894867,208,2.06032e-05,0.0147573,0.0297167,0.00481051,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
FBRQaFn4,large,150,ReLU(),0.010000,128,25,0.00010,0.000000,1.000000,0.000000,0.963214,0.00574963,0.00564066,179,3.7751e-07,0.0144633,0.0279768,0.00574926,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
iZFqxiQm,large,150,ReLU(),0.010000,128,25,0.00010,0.000000,1.000000,0.000000,0.973509,0.00414474,0.00369697,299,4.45046e-06,0.0260308,0.0302668,0.00414029,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
n7Rx5HAO,large,150,ReLU(),0.010000,128,25,0.00010,0.000000,1.000000,0.000000,0.960391,0.00619096,0.00575483,121,4.45438e-07,0.0230193,0.0266484,0.00619052,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
q5Wkjbo9,large,150,ReLU(),0.010000,128,25,0.00010,0.000000,1.000000,0.000000,0.96149,0.00602175,0.00569987,211,3.01062e-06,0.0203944,0.0314425,0.00601873,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=


# Figure: Effects of model parameters on reducibility

In [512]:
fig = plt.figure(figsize=(9,1.75))
gs = gridspec.GridSpec(1,4, figure=fig,wspace=.5)
plt.gcf().subplots_adjust(bottom=0.25)

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])




# Orthogonal models which have been reduced
df1 = pd.DataFrame((Model() & 'lambda_r=0' & 'lambda_o=1' & 'patience=50').proj('model_id','lambda_r','lambda_o','patience').fetch())
df = pd.DataFrame((LCA() ).proj('model_id','valid_loss').fetch())
orthogonal_models = pd.DataFrame(data={'model_id':np.random.choice(np.unique((LCA() & pd.DataFrame(df1['model_id'])).fetch('model_id')),25,replace=False)})

# Non-Orthogonal models which have been reduced
df1 = pd.DataFrame((Model() & 'lambda_r=0' & 'lambda_o=0' & 'patience=50').proj('model_id','lambda_r','lambda_o','patience').fetch())
df = pd.DataFrame((LCA() ).proj('model_id','valid_loss').fetch())
non_orthogonal_models = pd.DataFrame(data={'model_id':np.random.choice(np.unique((LCA() & pd.DataFrame(df1['model_id'])).fetch('model_id')),25,replace=False)})

# Concanenate models
models = orthogonal_models.append(non_orthogonal_models)

# Panel A
df1 = pd.DataFrame((Model() & models).proj('model_id','lambda_o','epochs','r2').fetch())
df = pd.DataFrame((LCA() ).proj('model_id','valid_loss').fetch())
#df = df.groupby('model_id')['valid_loss'].apply(np.min).reset_index()
df['valid_loss']=1-(df['valid_loss'].astype(float))
df = df.merge(df1,on='model_id',how='left')

sns.histplot(data=df,x='valid_loss',hue='lambda_o',ax=ax0,kde=True,stat='probability',legend=False)
ax0.set_ylabel('Probability', fontsize=8)
ax0.set_xlabel('Reducibility', fontsize=8)

# Panel B: Epochs and model performance

df = pd.DataFrame((LCA() ).proj('model_id','valid_loss','lr','r2_x','r2_xqt','r2_z').fetch())
df1 = pd.DataFrame((Model()  ).proj('model_id','lambda_r','r2','patience','epochs','l2_rate').fetch())
df['valid_loss']=1-(df['valid_loss'].astype(float))
df = df.merge(df1,on='model_id',how='left')
df['lambda_r'] = df['lambda_r'].astype(float)
df['r2_z'] =1-df['r2_z'].astype(float)
df['r2_x'] =1-df['r2_x'].astype(float)
df['r2_xqt'] =1-df['r2_xqt'].astype(float)

sns.scatterplot(data=df,x='epochs',y='r2',hue='patience',ax=ax1,legend=False,s=5)
ax1.set_ylabel('Model performance', fontsize=8)
ax1.set_xlabel('Epochs', fontsize=8)

# Panel C
sns.scatterplot(data=df,x='epochs',y='valid_loss',hue='patience',ax=ax2,legend=False,s=5)
ax2.set_ylabel('Reducibility', fontsize=8)
ax2.set_xlabel('Epochs', fontsize=8)


df = pd.DataFrame((LCA() ).proj('model_id','w_rec','q').fetch())

#df = df.merge(df1,on='model_id',how='left')
df1 = df.groupby(['model_id'])['q'].apply(np.mean).reset_index()
df2 = df.groupby(['model_id'])['w_rec'].apply(np.mean).reset_index()

df = df1.merge(df2,on='model_id',how='left')

df1 = pd.DataFrame((Model()).proj('model_id','w_rec','w_in','w_out','patience','lambda_r').fetch())
df = df.merge(df1,on='model_id',how='left')

def conjugate(row):
    w = row['w_rec_y']
    q = row['q']
    return q @ w @ q.T
from scipy.stats import pearsonr
def correlation(row):
    wbar = row['w_rec_x']
    conj = row['conjugate']
    corr,_ =pearsonr(wbar.flatten(),conj.flatten())
    return corr

def q_corr(row):
    w_in = row['w_in']
    w_out = row['w_out']
    q = row['q'].T
    w_ext = np.hstack((w_in,w_out.T))
    corr,_ =pearsonr(w_ext.flatten(),q.flatten())
    return corr



df['conjugate']=df.apply(conjugate,axis=1)
df['corr']=df.apply(correlation,axis=1)
df['q_corr']=df.apply(q_corr,axis=1)


# Panel D
sns.histplot(data=df,x='corr',ax=ax3,kde=True,stat='probability',hue='patience',legend=False)
ax3.set_ylabel('Probability', fontsize=8)
ax3.set_xlabel('Correlation between ' + r'$\bar{W}$' ' and ' +r'$Q^TWQ$', fontsize=8)



for ax in [ax0,ax1,ax2,ax3]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(6)
    ax.tick_params(axis='both', which='major', pad=1)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    
plt.savefig('/users/langdon/figures/circuitpaperfigures/model_effects.png',dpi=1000, transparent=True)

# Figure: Model mismatch

In [3]:
fig = plt.figure(figsize=(4,2))
gs = gridspec.GridSpec(1,2, figure=fig,wspace=.5)
plt.gcf().subplots_adjust(bottom=0.25)

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])



df = pd.DataFrame((LCA() ).proj('model_id','w_rec','q').fetch())

#df = df.merge(df1,on='model_id',how='left')
df1 = df.groupby(['model_id'])['q'].apply(np.mean).reset_index()
df2 = df.groupby(['model_id'])['w_rec'].apply(np.mean).reset_index()

df = df1.merge(df2,on='model_id',how='left')

df1 = pd.DataFrame((Model()).proj('model_id','w_rec','w_in','w_out','patience','lambda_r').fetch())
df = df.merge(df1,on='model_id',how='left')

def conjugate(row):
    w = row['w_rec_y']
    q = row['q']
    return q @ w @ q.T
from scipy.stats import pearsonr
def correlation(row):
    wbar = row['w_rec_x']
    conj = row['conjugate']
   
    corr,_ =pearsonr(wbar.flatten(),conj.flatten())
    return corr

def q_corr(row):
    w_in = row['w_in']
    w_out = row['w_out']
    q = row['q'].T
    w_ext = np.hstack((w_in,w_out.T))
    corr,_ =pearsonr(w_ext.flatten(),q.flatten())
    return corr



df['conjugate']=df.apply(conjugate,axis=1)
df['corr']=df.apply(correlation,axis=1)
df['q_corr']=df.apply(q_corr,axis=1)


# Panel A
sns.histplot(data=df,x='corr',ax=ax1,kde=True,stat='probability')
ax0.set_ylabel('Probability', fontsize=8)
ax0.set_xlabel( r'$R^2$', fontsize=8)
# Panel B
df = pd.DataFrame(LCA().fetch())
df['score'] = 1-df['r2']
sns.histplot(data=df,x='score',ax=ax0,kde=True,stat='probability')
ax1.set_ylabel('Probability', fontsize=8)
ax1.set_xlabel( r'$Corr(\bar{W},Q^TWQ)$', fontsize=8)

for ax in [ax0,ax1]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(6)
    ax.tick_params(axis='both', which='major', pad=1)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    
plt.savefig('/users/langdon/figures/circuitpaperfigures/mismatch.png',dpi=1000, transparent=True)

In [9]:
LCA()

model_id unique model id,lca_id,alpha,sigma_rec,activation,lr,weight_decay,patience,threshold,n_trials,batch_size,max_epochs,epochs,r2,r2_x,r2_xqt,r2_z,valid_loss,train_loss,valid_loss_history,train_loss_history,w_rec,w_in,w_out,q,a,w_error,q_error
04XOWLZl,1SJWYgkP,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,263,0.1196,0.0569,0.1009,0.1494,0.11967,0.12056,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
04XOWLZl,J4tQKQ9h,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,349,0.1076,0.0575,0.0908,0.1119,0.10756,0.11606,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
04XOWLZl,LCHeFXV2,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,240,0.1054,0.0556,0.0858,0.1181,0.10537,0.10515,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
04XOWLZl,mDYXgKZb,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,207,0.1042,0.0565,0.0852,0.1133,0.10422,0.10405,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
04XOWLZl,QuoWj44i,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,188,0.1125,0.0602,0.0940,0.1213,0.11251,0.11595,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
08Qmt6aZ,5t2wWnpA,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,244,0.0726,0.0189,0.0745,0.1409,0.07267,0.06981,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
08Qmt6aZ,havfZAZ4,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,306,0.0633,0.0189,0.0582,0.1085,0.06329,0.06362,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
08Qmt6aZ,hOQNH699,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,197,0.0822,0.0200,0.0921,0.1478,0.08219,0.08426,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
08Qmt6aZ,lcRA2sZe,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,336,0.0616,0.0182,0.0598,0.1093,0.06163,0.06184,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
08Qmt6aZ,ZrvNiYFg,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,219,0.0777,0.0197,0.0861,0.1337,0.07770,0.08280,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan


In [498]:
df.sort_values('corr',ascending=False)

,model_id,q,w_rec_x,patience,lambda_r,w_rec_y,w_in,w_out,conjugate,corr,q_corr
186,PU3GbIco,"[[0.0005462527, 0.0063869744, -0.023979664, 0....","[[0.3930877, -1.0392449, 0.25192112, 0.2216548...",10,0.000000,"[[0.0, 0.02029034, 0.01580903, 0.018631574, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0....","[[0.28969398, 0.0, 0.00020398837, 0.0, 0.0, 0....","[[0.49958113, -0.9801012, -0.034751445, 0.0446...",0.896172,0.580524
44,7MGKnyFO,"[[0.002075263, -0.0033248875, 0.017473256, -0....","[[0.41262147, -0.97943586, -0.12235451, -0.179...",25,0.010000,"[[0.0, 0.24002185, 0.0, 0.3010143, 0.0, 3.2441...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.64005595], [0.0, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.3820895, -0.7620392, -0.44118363, -0.42844...",0.893387,0.496453
224,U6Q1AIZk,"[[0.0032683224, 0.0030580207, -0.011229684, 0....","[[0.41362652, -0.75076604, -0.13966195, -0.296...",50,0.100000,"[[0.0, 0.014313156, 0.011194335, 0.0047627026,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.7781568], [0.0, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.50813746, -1.1620005, -0.40776312, -0.8038...",0.892308,0.511609
202,RKZKoXGF,"[[0.010181147, -0.0017462864, 0.004652772, 0.0...","[[0.5048603, -0.51487124, 0.11101496, 0.005474...",50,0.000000,"[[0.0, 0.00512342, 0.012008795, 0.0057568736, ...","[[0.0, 0.0, 0.0, 0.8972955, 0.0, 0.0], [0.0, 0...","[[0.0, 0.0, 0.0, 0.0, 0.6172393, 0.0, 0.0, 0.0...","[[0.66750664, -0.72632617, -0.03657019, -0.803...",0.886814,0.418065
104,FsmloIe3,"[[-0.0005973041, 0.0030066713, -0.0052370536, ...","[[0.36699262, -1.4741405, 0.31826103, -0.31289...",25,0.100000,"[[0.0, 0.0047709625, 0.0142561365, 0.022189999...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0....","[[0.29755622, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[[0.43117818, -1.5997791, -0.14675672, -0.7243...",0.883344,0.552452
...,...,...,...,...,...,...,...,...,...,...,...
36,66cfLnn0,"[[-0.112641856, -0.04117283, -0.079851165, -0....","[[0.3720481, 0.65421224, -3.457475, -3.2622344...",100,0.100000,"[[0.0, 0.0, 0.054224573, 0.021993864, 0.0, 0.0...","[[0.0, 0.0, 1.3510923, 0.0, 0.0, 0.0], [0.0, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 2.1643724, 1.704940...","[[1.2805258, 0.8585138, -0.64691985, -0.839462...",0.339171,0.359141
239,W21awspy,"[[-0.003957397, -0.003195548, 0.00039886226, -...","[[0.21581176, -1.2116632, 0.14488088, -0.14810...",50,0.000000,"[[0.0, 0.0039299536, 0.003933766, 8.897298e-06...","[[0.035210297, 0.05235598, 0.05682565, 0.08700...","[[0.028603474, 0.042607293, 0.03725142, 0.0272...","[[0.21520074, -0.8981915, -0.51323664, -0.7411...",0.294411,0.532892
74,B8HkhZLQ,"[[0.0025206148, 0.0053572967, 0.0005961925, -0...","[[0.2734936, -0.8356103, 0.14443882, -0.520975...",50,0.000000,"[[0.0, 0.0042602923, 6.615209e-05, 0.001100593...","[[0.08095528, 0.08086469, 0.10561077, 0.032373...","[[0.04057501, 0.035379443, 0.06316734, 0.03950...","[[0.5742789, -1.5061888, -1.256633, -0.2934999...",0.285857,0.538305
165,Mw5j5mFA,"[[-0.0042542457, -0.0008018523, 0.0027384576, ...","[[0.12603858, -0.4960564, -0.00019389093, -0.0...",50,0.000000,"[[0.0, 0.0027009672, 0.0055855983, 0.000473806...","[[0.081940524, 0.08250294, 0.03258069, 0.07398...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.365572, -0.9531441, -1.1003668, -0.3447178...",0.230742,0.598541


# Figure: Sloppiness

In [206]:
pd.DataFrame((LCA() & 'model_id="0DJahE7H"').fetch()).sort_values('valid_loss').head(5)

,model_id,lca_id,alpha,sigma_rec,activation,lr,weight_decay,patience,threshold,n_trials,...,train_loss,valid_loss_history,train_loss_history,w_rec,w_in,w_out,q,a,w_error,q_error
12,0DJahE7H,8OqAUOwV,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,...,0.04225,"[0.40309122644380996, 0.3489719340961564, 0.26...","[0.5746754216423386, 0.379392439557105, 0.2978...","[[0.6293925, -0.49713117, -0.06357691, 0.23688...","[[0.9695663, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.29149684, 0....","[[0.0027772486, 0.001981385, 0.008796111, 0.00...","[[0.86408633, 0.5494416, 0.37610477, 0.2213613...",NaN,NaN
3,0DJahE7H,2bJxAKpb,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,...,0.04104,"[0.40937866383674604, 0.344350755667221, 0.265...","[0.567739790522787, 0.37257031697781917, 0.284...","[[0.5927951, -0.5263529, -0.097258046, 0.31606...","[[1.0665934, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.29584247, 0....","[[-0.0016542077, 0.001641009, 0.0063456446, -0...","[[0.765721, 1.0005748, 0.67484087, 0.8030972, ...",NaN,NaN
86,0DJahE7H,v8x49pfK,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,...,0.04613,"[0.40244958832569083, 0.3588565476157919, 0.28...","[0.5739902481881419, 0.37861706158808245, 0.31...","[[0.6585896, -0.36347625, -1.113707, -0.600687...","[[0.9227021, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3122384, 0.0...","[[-0.00078146905, 0.0022945106, 0.011056781, 0...","[[0.18595447, 0.363713, 0.76698315, 0.6706305,...",NaN,NaN
47,0DJahE7H,hk9CtbcW,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,...,0.04526,"[0.40884185399771256, 0.35046655043086883, 0.2...","[0.5675439545141894, 0.3784348454510331, 0.293...","[[0.48551464, -0.8472268, -0.39633292, -0.1297...","[[1.0728697, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.31141138, 0....","[[0.0011548698, 0.00848094, 0.007480979, 0.007...","[[0.797933, 0.9982965, 0.56408525, 0.24810359,...",NaN,NaN
28,0DJahE7H,cHGhxswV,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,...,0.04388,"[0.40681713761321375, 0.3514601563683301, 0.28...","[0.5823078301375932, 0.3807144715484561, 0.309...","[[0.5109194, -3.1336467, 0.025151445, -0.43488...","[[1.0464588, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28887087, 0....","[[0.0012663603, 0.006849408, 0.011975491, 0.00...","[[0.22747743, 0.6373319, 0.6531899, 0.37559807...",NaN,NaN


In [5]:
fig = plt.figure(figsize=(10,3.25))
gs = gridspec.GridSpec(2,5, figure=fig,width_ratios=[1,1,1.25,1,1],wspace=.5,hspace=.5)
plt.gcf().subplots_adjust(bottom=0.15)

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])
ax4 = fig.add_subplot(gs[4])

ax5 = fig.add_subplot(gs[5])
ax6 = fig.add_subplot(gs[6])
ax7 = fig.add_subplot(gs[7])
ax8 = fig.add_subplot(gs[8])
ax9 = fig.add_subplot(gs[9])

model_id = "7MGKnyFO"

# Panel 0
cond_frame = pd.DataFrame(data={'model_id':[ model_id] })
df = pd.DataFrame((LCA() & cond_frame & 'lr=.02' ).proj('model_id','lr','valid_loss','w_rec','w_in','w_out','q').fetch())
df['valid_loss']=1-df['valid_loss'].astype(float)
#percentile = np.percentile(df['valid_loss'],50)
#df = df[df['valid_loss']>percentile]
sns.histplot(data=df,x='valid_loss',ax=ax0,edgecolor='white',linewidth=.25,legend=False)
#ax1.set_title('Distribution of loss (within model)')
ax0.set_xlabel('$R^2$',fontsize=8,labelpad=-1)
ax0.set_ylabel('Count',labelpad=1,fontsize=8)
ax0.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))

# Panel 1
ax1.set_xlabel('Principal component',fontsize=8, labelpad=1)
w_recs = np.stack(df['w_rec']).reshape(-1,8*8)
w_ins = np.stack(df['w_in']).reshape(-1,8*6)
w_outs = np.stack(df['w_out']).reshape(-1,2*8)
qs = np.stack(df['q']).reshape(-1,150*8)
ws = np.concatenate((w_ins,w_recs,w_outs,qs),axis=1)
pca1 = PCA()
x=pca1.fit_transform(ws)
ax1.plot(pca1.explained_variance_ratio_[:25],marker='.',alpha=1,label='Model one',markersize=5)
ax1.set_ylabel('Variance ',fontsize=8)

# Panel 2 (PCA scatter)
df1 = pd.DataFrame(data={'pc1':x[:,0],'pc2':x[:,1],'pc10':x[:,10]})
sns.scatterplot(data=df1,x='pc1',y='pc2',alpha=.5,ax=ax2,s=5, color='k')
sns.kdeplot(data=df1,x='pc1',y='pc2',alpha=.25,ax=ax2,fill=True,levels=100)
ax2.set_xlabel('PC 1 '+ '{:.1f}'.format(100*pca1.explained_variance_ratio_[0]) + ' % ',fontsize=8, labelpad=1)
ax2.set_ylabel('PC 2 '+ '{:.1f}'.format(100*pca1.explained_variance_ratio_[1]) + ' % ',fontsize=8, labelpad=-1)
#ax4.set_title('Distribution of latent parameters')

# Panel 3
gs3 = gs[3].subgridspec(2, 3,height_ratios=[1,.25],width_ratios=[1,.25,.25])
ax300=fig.add_subplot(gs3[0,0])
ax301=fig.add_subplot(gs3[0,1])
ax302=fig.add_subplot(gs3[:2,2])
ax310=fig.add_subplot(gs3[1,0])

y = pca1.components_
w_in = y[0,:8*6].reshape(8,6)
w_rec = y[0,8*6:8*6+8*8].reshape(8,8)
w_out = y[0,8*6+8*8:8*6+8*8+ 2*8].reshape(2,8)
q = y[0,8*6+8*8+2*8:].reshape(150,8)
min=np.min(y[0,:])
max=np.max(y[0,:])
sns.heatmap(w_in,ax=ax301,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(w_rec,ax=ax300,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(w_out,ax=ax310,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(q,ax=ax302,center=0,vmin=-1,vmax=1,cbar=False,xticklabels=False,yticklabels=False)
ax3.set_title('PC 1',fontsize=9)

# Panel 4 (Mean solution)
df = pd.DataFrame((LCA() & {'model_id':model_id}).fetch())
gs4 = gs[4].subgridspec(2, 3,height_ratios=[1,.25],width_ratios=[1,.25,.25])
ax400=fig.add_subplot(gs4[0,0])
ax401=fig.add_subplot(gs4[0,1])
ax402=fig.add_subplot(gs4[:2,2])
ax410=fig.add_subplot(gs4[1,0])
min=None
max=None
sns.heatmap(np.mean(df['w_in']),ax=ax401,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(np.mean(df['w_rec']),ax=ax400,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(np.mean(df['w_out']),ax=ax410,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(np.mean(df['q']).T,ax=ax402,center=0,vmin=-1,vmax=1,cbar=False,xticklabels=False,yticklabels=False)


# Bottom row
# Panel 5
#cond_frame = pd.DataFrame(data={'model_id':[ model_id] })
df = pd.DataFrame((LCA() & (Model()& 'patience=50' & 'lambda_o=1' & 'lambda_r=0').proj('model_id') ).fetch())
df.valid_loss = ( 1-df.valid_loss).astype(float)
#percentile = np.percentile(df['valid_loss'],95)
sns.histplot(data=df,x='valid_loss',ax=ax5,edgecolor='white',linewidth=.25,legend=False)
#ax1.set_title('Distribution of loss (within model)')
ax5.set_xlabel('$R^2$',fontsize=8,labelpad=-1)
ax5.set_ylabel('Count',labelpad=1,fontsize=8)
ax5.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))


# Panel 6
df = pd.DataFrame((LCA() & (Model()& 'patience=25' & 'lambda_o=1' & 'lambda_r=0.01').proj('model_id') ).proj('w_rec','w_in','w_out').fetch())
df_rec = df.groupby(['model_id'])['w_rec'].apply(np.mean).reset_index()
df_in = df.groupby(['model_id'])['w_in'].apply(np.mean).reset_index()
df_out = df.groupby(['model_id'])['w_out'].apply(np.mean).reset_index()
df1 = df_rec.merge(df_in,on='model_id',how='left')
df2 = df1.merge(df_out,on='model_id',how='left')

w_recs = np.stack(df2['w_rec']).reshape(-1,8*8)
w_ins = np.stack(df2['w_in']).reshape(-1,8*6)
w_outs = np.stack(df2['w_out']).reshape(-1,2*8)


ws = np.concatenate((w_ins,w_recs,w_outs),axis=1)
pca1 = PCA()
x=pca1.fit_transform(ws)
ax6.plot(pca1.explained_variance_ratio_[:25],marker='.',alpha=1,label='Model one',markersize=5)
ax6.set_ylabel('Variance ',fontsize=8)
ax6.set_xlabel('Principal component',fontsize=8, labelpad=1)

# Panel 7
df1 = pd.DataFrame(data={'pc1':x[:,0],'pc2':x[:,1],'pc10':x[:,10]})
sns.scatterplot(data=df1,x='pc1',y='pc2',alpha=.5,ax=ax7,s=5, color='k')
sns.kdeplot(data=df1,x='pc1',y='pc2',alpha=.25,ax=ax7,fill=True,levels=100)
ax7.set_xlabel('PC 1 '+ '{:.1f}'.format(100*pca1.explained_variance_ratio_[0]) + ' % ',fontsize=8, labelpad=1)
ax7.set_ylabel('PC 2 '+ '{:.1f}'.format(100*pca1.explained_variance_ratio_[1]) + ' % ',fontsize=8, labelpad=-1)
#ax4.set_title('Distribution of latent parameters')

# Panel 8
gs8 = gs[8].subgridspec(2, 3,height_ratios=[1,.25],width_ratios=[1,.25,.25])
ax800=fig.add_subplot(gs8[0,0])
ax801=fig.add_subplot(gs8[0,1])
ax810=fig.add_subplot(gs8[1,0])

y = pca1.components_
w_in = y[0,:8*6].reshape(8,6)
w_rec = y[0,8*6:8*6+8*8].reshape(8,8)
w_out = y[0,8*6+8*8:8*6+8*8+ 2*8].reshape(2,8)

min=np.min(y[0,:])
max=np.max(y[0,:])
sns.heatmap(w_in,ax=ax801,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(w_rec,ax=ax800,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(w_out,ax=ax810,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
ax8.set_title('PC 1',fontsize=9)

# Panel 9 Mean solution


gs9 = gs[9].subgridspec(2, 3,height_ratios=[1,.25],width_ratios=[1,.25,.25])
ax900=fig.add_subplot(gs9[0,0])
ax901=fig.add_subplot(gs9[0,1])

ax910=fig.add_subplot(gs9[1,0])
min=None
max=None
sns.heatmap(np.mean(df2['w_in']),ax=ax901,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(np.mean(df2['w_rec']),ax=ax900,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(np.mean(df2['w_out']),ax=ax910,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
#sns.heatmap(np.mean(df['q']).T,ax=ax902,center=0,vmin=-1,vmax=1,cbar=False,xticklabels=False,yticklabels=False)



for ax in [ax3,ax4,ax8,ax9]:
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.tick_params(left=False)
    ax.tick_params(bottom=False)


for ax in [ax0,ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(6)
    ax.tick_params(axis='both', which='major', pad=1)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    
lims = [
            np.min([ax3.get_xlim(), ax3.get_ylim()]),  # min of both axes
            np.max([ax3.get_xlim(), ax3.get_ylim()]),  # max of both axes
            ]

ax3.set_aspect('equal')
#ax3.set_xlim(lims)
#ax3.set_ylim(lims)

plt.savefig('/users/langdon/figures/circuitpaperfigures/sloppiness.png',dpi=1000, transparent=True)

# Figure: Orthogonality and correlations

In [262]:
LCA_unconstrained()

model_id unique model id,lca_id,lca2_id,alpha,sigma_rec,lr,weight_decay,patience,threshold,n_trials,batch_size,max_epochs,epochs,r2_x,r2_xqt,r2_z,valid_loss,train_loss,valid_loss_history,train_loss_history,w_rec,w_in,w_out,q,a,w_error,q_error
1CE85dgv,0w8KiPd8,IYgFXeLn,0.20,0.15,0.0100000,0.00000,25,0.0001000,64,128,10000,154,0.0142,0.0362,-0.0161,0.03428,0.03513,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
1CE85dgv,1UB1vOi6,U0YLsQuf,0.20,0.15,0.0100000,0.00000,25,0.0001000,64,128,10000,150,0.0137,0.0320,-0.0145,0.03122,0.03226,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
1CE85dgv,lS3IaKc6,7poLUkU9,0.20,0.15,0.0100000,0.00000,25,0.0001000,64,128,10000,97,0.0144,0.0463,-0.0198,0.04086,0.03590,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
1CE85dgv,pDuw8KSZ,xRcyZR6L,0.20,0.15,0.0100000,0.00000,25,0.0001000,64,128,10000,236,0.0133,0.0290,-0.0131,0.02920,0.02995,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
1CE85dgv,rqqmUfgx,OJxtfo3W,0.20,0.15,0.0100000,0.00000,25,0.0001000,64,128,10000,190,0.0141,0.0326,-0.0150,0.03166,0.03307,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
88vRqebc,eI6vX0BY,1iAKoXCu,0.20,0.15,0.0100000,0.00000,25,0.0001000,64,128,10000,193,0.0166,0.0241,-0.0113,0.02944,0.02930,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
88vRqebc,Hq6sE0C9,k0MKkAKi,0.20,0.15,0.0100000,0.00000,25,0.0001000,64,128,10000,178,0.0168,0.0244,-0.0112,0.02999,0.02951,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
88vRqebc,LGS6pT1x,8prt9aqg,0.20,0.15,0.0100000,0.00000,25,0.0001000,64,128,10000,124,0.0187,0.0333,-0.0144,0.03767,0.03753,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
88vRqebc,NoQ5Ey8M,Qbp8PBTg,0.20,0.15,0.0100000,0.00000,25,0.0001000,64,128,10000,145,0.0182,0.0321,-0.0143,0.03598,0.03721,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
88vRqebc,wRspO5j3,O75TerCq,0.20,0.15,0.0100000,0.00000,25,0.0001000,64,128,10000,247,0.0158,0.0228,-0.0103,0.02835,0.02929,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan


In [4]:
fig = plt.figure(figsize=(3.75,3.5))
gs = gridspec.GridSpec(2,2, figure=fig,wspace=.5,hspace=.5)
plt.gcf().subplots_adjust(bottom=0.15)
ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])

# Panel C: Effect of diagonality constraint on reduction loss
df1 = pd.DataFrame((LCA() & (Model() & 'lambda_o=0').proj('model_id')).proj('valid_loss','w_rec','w_in','w_out').fetch())
df2 = pd.DataFrame(LCA_unconstrained().proj('valid_loss','w_rec','w_in','w_out').fetch())
df1.valid_loss = 1-df1.valid_loss
df2.valid_loss = 1-df2.valid_loss
sns.histplot(data=df1,x='valid_loss',ax=ax2,kde=True,stat='probability')
sns.histplot(data=df2,x='valid_loss',ax=ax2,color='orange',kde=True,stat='probability')

model_id="JjoAB132"
lca2_id = "t0wQBQvz"
vmin=.1
vmax=.6
# Panel A: Example of W_extT W_ext

w_in = (Model() & {'model_id': model_id}).fetch1('w_in')
w_out = (Model() & {'model_id': model_id}).fetch1('w_out')
w_ext = np.hstack((w_in,w_out.T))
w_ext = w_ext / np.linalg.norm(w_ext,axis=0)
corr = w_ext.T@w_ext
mask = np.triu(np.ones_like(w_ext.T@w_ext, dtype=bool))
sns.heatmap(w_ext.T @w_ext,ax=ax0,cmap='flare_r',mask=mask,cbar=True,linewidths=.5,vmin=vmin,vmax=vmax, xticklabels = [r'$i_1$',
                    r'$i_2$',
                   r'$i_3$',
                   r'$i_4$',
                   r'$i_5$',
                   r'$i_6$',
                   r'$o_1$',
                   r'$o_2$'], yticklabels = [r'$i_1$',
                    r'$i_2$',
                   r'$i_3$',
                   r'$i_4$',
                   r'$i_5$',
                   r'$i_6$',
                   r'$o_1$',
                   r'$o_2$'])

# Panel B:
w_in = (LCA_unconstrained() & {'lca2_id': lca2_id}).fetch1('w_in')
w_out = (LCA_unconstrained() & {'lca2_id': lca2_id}).fetch1('w_out')
w_ext = np.hstack((w_in,w_out.T))
w_ext = w_ext / np.linalg.norm(w_ext,axis=0)
mask = np.triu(np.ones_like(w_ext.T@w_ext, dtype=bool))
sns.heatmap(w_ext.T @w_ext,ax=ax1,cmap='flare_r',cbar=True,linewidths=.5,mask=mask,vmin=vmin,vmax=vmax)

# Panel D: Inheritance of correlations
def inheritance(row):

    w_in = row['w_in_x']
    w_out = row['w_out_x']
    w_ext = np.hstack((w_in,w_out.T))
    #w_ext = w_ext / np.linalg.norm(w_ext,axis=0)
    corr_1 = w_ext.T @ w_ext
    
    w_in = row['w_in_y']
    w_out = row['w_out_y']
    w_ext = np.hstack((w_in,w_out.T))
    #w_ext = w_ext / np.linalg.norm(w_ext,axis=0)
    corr_2 = w_ext.T @ w_ext

    r,_ = stats.pearsonr(np.triu(corr_1).flatten(),np.triu(corr_2).flatten())
    return r

df1 = pd.DataFrame((Model() & LCA_unconstrained().proj('model_id')).proj('r2','w_rec','w_in','w_out').fetch())
df2 = pd.DataFrame(LCA_unconstrained().proj('valid_loss','w_rec','w_in','w_out').fetch())
df = df2.merge(df1,on='model_id',how='left')
df['inheritances'] = df.apply(lambda x: inheritance(x), axis=1)
sns.histplot(data=df,x='inheritances',ax=ax3,kde=True,stat='probability')


cbar = ax0.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)
cbar = ax1.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)

ax1.tick_params(left=False)
ax1.tick_params(bottom=False)
ax1.set_xticklabels([])
ax1.set_yticklabels([])

for ax in [ax0]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(8)
    #ax.set_xticklabels([])
    #ax.set_yticklabels([])
#     ax.spines["top"].set_visible(False)
#     ax.spines["right"].set_visible(False)
#     ax.spines["left"].set_visible(False)
#     ax.spines["bottom"].set_visible(False)
    #ax.tick_params(left=False)
    #ax.tick_params(bottom=False)
# ax0.set_xticklabels([r'$i_1$',
#                     r'$i_2$',
#                    r'$i_3$',
#                    r'$i_4$',
#                    r'$i_5$',
#                    r'$i_6$',
#                    r'$o_1$',
#                    r'$o_2$'],fontsize=4)
#cbar = ax1.collections[0].colorbar
#cbar.ax.tick_params(labelsize=8)
#ax.tick_params(bottom=False)

for ax in [ax2,ax3]:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    #ax.tick_params(left=False)
    #ax.tick_params(bottom=False)
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(6)

ax0.set_title(r'$W_{ext}^TW_{ext}$',fontsize=8)
#ax0.set_xlabel('Columns of ' + r'$W_{ext}$',fontsize=8,labelpad=-5)
#ax0.set_ylabel('Columns of ' + r'$W_{ext}$',fontsize=8,labelpad=-5)

ax1.set_title(r'$\bar{W}_{ext}^T\bar{W}_{ext}$',fontsize=8)
#ax1.set_xlabel('Columns of ' + r'$W_{ext}$',fontsize=8,labelpad=-5)
#ax1.set_ylabel('Columns of ' + r'$W_{ext}$',fontsize=8,labelpad=-5)

ax2.set_xlabel(r'$R^2$',fontsize=8, labelpad=1)
ax2.set_ylabel('Probability',fontsize=8, labelpad=1)

ax3.set_xlabel(r'$\rho$',fontsize=8, labelpad=1)
ax3.set_ylabel('Probability',fontsize=8, labelpad=1)
plt.savefig('/users/langdon/figures/circuitpaperfigures/orthogonality.png',dpi=1000, transparent=True)

In [18]:
df.sort_values('inheritances', ascending=False)

,model_id,lca_id,lca2_id,valid_loss,w_rec_x,w_in_x,w_out_x,r2,w_rec_y,w_in_y,w_out_y,inheritances
99,W21awspy,lTwSUrmy,0bVMZrpg,0.03899,"[[0.080901094, -0.9734933, -0.041888762, -0.32...","[[0.78639656, 0.0, 0.11315403, 0.11814278, 0.0...","[[0.0, 0.0, 0.45938313, 0.0071162055, 0.217303...",0.964686,"[[0.0, 0.0039299536, 0.003933766, 8.897298e-06...","[[0.035210297, 0.05235598, 0.05682565, 0.08700...","[[0.028603474, 0.042607293, 0.03725142, 0.0272...",0.976446
98,W21awspy,fv266NJB,d2wLFMTM,0.03641,"[[0.10632027, -0.9947414, -0.049213566, -0.353...","[[0.7750777, 0.0037712324, 0.08679595, 0.14685...","[[0.0, 0.0, 0.44097143, 0.00835853, 0.23914522...",0.964686,"[[0.0, 0.0039299536, 0.003933766, 8.897298e-06...","[[0.035210297, 0.05235598, 0.05682565, 0.08700...","[[0.028603474, 0.042607293, 0.03725142, 0.0272...",0.973002
89,tN4IsMyf,pmSHkK2Y,uzoVFy4A,0.02967,"[[0.012989606, -3.4045155, -0.24932021, -0.035...","[[1.0664167, 0.010914746, 0.0, 0.21830902, 0.0...","[[0.073347494, 0.036032557, 0.35179833, 0.0, 0...",0.984465,"[[0.0, 5.2150972e-05, 1.4267345e-05, 6.3317006...","[[0.0129888235, 0.014277137, 0.017463282, 0.01...","[[0.08457772, 0.044298105, 0.068052605, 0.0694...",0.970743
97,W21awspy,bDjJNfJ4,742NS531,0.04342,"[[0.1835323, -0.98367745, -0.07962871, -0.1442...","[[0.7788491, 0.03061049, 0.09053951, 0.1952741...","[[0.0001831927, 0.0, 0.48650095, 0.032128613, ...",0.964686,"[[0.0, 0.0039299536, 0.003933766, 8.897298e-06...","[[0.035210297, 0.05235598, 0.05682565, 0.08700...","[[0.028603474, 0.042607293, 0.03725142, 0.0272...",0.969253
40,JjoAB132,A8dCJ6Il,t0wQBQvz,0.02783,"[[0.21664271, -1.0315645, 0.025835827, 0.16283...","[[0.863694, 0.4298624, 0.0, 0.0042594494, 0.00...","[[0.0, 0.0, 0.3364694, 0.06490429, 0.14299178,...",0.979609,"[[0.0, 3.1877684e-05, 0.0013701196, 6.688541e-...","[[0.074206315, 0.05816797, 0.043124355, 0.0640...","[[0.0, 0.018543279, 0.0041357484, 0.01423514, ...",0.966089
...,...,...,...,...,...,...,...,...,...,...,...,...
20,bWCFg1EN,0rbmoXht,735E61J5,0.04229,"[[0.51079005, -2.4384785, 0.066398874, 0.52668...","[[0.64663005, 0.22133304, 0.010200175, 0.60790...","[[0.09193426, 0.0, 0.3347822, 0.10546708, 0.56...",0.962900,"[[0.0, 9.636245e-06, 5.370582e-06, 0.003982139...","[[0.035540063, 0.052090533, 0.059733294, 0.041...","[[0.058995347, 0.057018027, 0.064810865, 0.043...",0.823960
24,bWCFg1EN,OBh24jJP,GJpJiADB,0.03796,"[[0.47033808, -2.0518482, -0.13618772, 0.21557...","[[0.8302102, 0.19360898, 0.010077544, 0.595314...","[[0.11679947, 0.0, 0.40277418, 0.022576397, 0....",0.962900,"[[0.0, 9.636245e-06, 5.370582e-06, 0.003982139...","[[0.035540063, 0.052090533, 0.059733294, 0.041...","[[0.058995347, 0.057018027, 0.064810865, 0.043...",0.809464
51,LbF2high,8OLYs4fz,tRe1ieaM,0.04931,"[[-1.248049, -5.742109, -4.9522567, 0.78447497...","[[3.329121, 0.12093671, 0.0032899939, 0.194164...","[[0.07117002, 0.0, 0.36791885, 0.0, 0.58023256...",0.976973,"[[0.0, 0.003113564, 0.00066137535, 0.003224913...","[[0.10099229, 0.14400366, 0.047098767, 0.09612...","[[0.0019538358, 0.012223898, 0.0, 0.020063734,...",0.769790
72,OCjoMvRi,T2mNEMXc,4sRFbW9c,0.04074,"[[-2.271315, -15.90832, 0.21152563, 1.9688935,...","[[2.3104336, 8.845117e-05, 0.0, 0.15250558, 0....","[[0.013668828, 0.113108486, 0.46222013, 0.0993...",0.947550,"[[0.0, 6.317117e-05, 3.237326e-05, 2.7095102e-...","[[0.009238236, 0.010121034, 0.023697076, 0.010...","[[0.07004629, 0.10572157, 0.05299103, 0.070182...",0.763881


In [30]:
model_id="JjoAB132"
lca2_id = "t0wQBQvz"
# Panel A: Example of W_extT W_ext

w_in = (Model() & {'model_id': model_id}).fetch1('w_in')
w_out = (Model() & {'model_id': model_id}).fetch1('w_out')
w_ext = np.hstack((w_in,w_out.T))
w_ext = w_ext / np.linalg.norm(w_ext,axis=0)
corr1 = w_ext.T@w_ext


# Panel B:
w_in = (LCA_unconstrained() & {'lca2_id': lca2_id}).fetch1('w_in')
w_out = (LCA_unconstrained() & {'lca2_id': lca2_id}).fetch1('w_out')
w_ext = np.hstack((w_in,w_out.T))
w_ext = w_ext / np.linalg.norm(w_ext,axis=0)
corr2 = w_ext.T@w_ext
plt.figure()
plt.scatter(np.triu(corr1).flatten(),np.triu(corr2).flatten())



# Figure: Effect of patience on reducibility

In [93]:
fig = plt.figure(figsize=(5,4))
gs = gridspec.GridSpec(2,2, figure=fig,wspace=.5,hspace=.5)
ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])
plt.gcf().subplots_adjust(bottom=0.15)

df = pd.DataFrame((LCA() ).proj('model_id','valid_loss','lr','r2_x','r2_xqt','r2_z').fetch())
df1 = pd.DataFrame((Model()  ).proj('model_id','lambda_r','r2','patience','epochs','l2_rate').fetch())
df['valid_loss']=1-(df['valid_loss'].astype(float))
df = df.merge(df1,on='model_id',how='left')
df['lambda_r'] = df['lambda_r'].astype(float)
df['r2_z'] =1-df['r2_z'].astype(float)
df['r2_x'] =1-df['r2_x'].astype(float)
df['r2_xqt'] =1-df['r2_xqt'].astype(float)

sns.scatterplot(data=df,x='epochs',y='r2',hue='patience',ax=ax0,legend=False,s=5)
sns.scatterplot(data=df,x='epochs',y='valid_loss',hue='patience',ax=ax1,legend=False,s=5)
sns.scatterplot(data=df,x='l2_rate',y='r2',hue='lambda_r',ax=ax2,legend=False,s=5)
sns.scatterplot(data=df,x='l2_rate',y='valid_loss',hue='lambda_r',ax=ax3,legend=False,s=5)

for ax in [ax0,ax1,ax2,ax3]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(6)
    #ax.tick_params(axis='both', which='major', pad=0)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

ax0.set_ylabel('Performance',fontsize=8,labelpad=0)
ax0.set_xlabel('Epochs',fontsize=8,labelpad=-1)

ax1.set_ylabel('Reducibility',fontsize=8,labelpad=0)
ax1.set_xlabel('Epochs',fontsize=8,labelpad=-.5)

ax2.set_ylabel('Performance',fontsize=8,labelpad=0)
ax2.set_xlabel(r'$\Omega_r$',fontsize=8,labelpad=-.5)

ax3.set_ylabel('Reducibility',fontsize=8,labelpad=0)
ax3.set_xlabel(r'$\Omega_r$',fontsize=8,labelpad=-.5)

cbar = ax0.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)
cbar = ax1.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)

plt.savefig('/users/langdon/figures/circuitpaperfigures/patience.png',dpi=1000, transparent=True)


sns.set(font_scale=2)

AttributeError: 'NoneType' object has no attribute 'ax'

# Figure: Connectivity

In [111]:

model_id = "7MGKnyFO"
#model_id = (LCA() & {'lca_id': lca_id}).fetch1('model_id')
lca_data = (LCA() & {'model_id':model_id}).fetch()
linewidth=1.5



fig = plt.figure(figsize=(8,1.75))
gs = gridspec.GridSpec(1,4, figure=fig,width_ratios=[1,1,.5,1],wspace=.5)
plt.gcf().subplots_adjust(top=0.825)
plt.gcf().subplots_adjust(bottom=0.15)
# RNN connectivity
ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])


# Panel A
gs0 = gs[0].subgridspec(2, 2, height_ratios=[150,2],width_ratios=[150,6],hspace=.1,wspace=.1)
ax000=fig.add_subplot(gs0[0,0])
ax001=fig.add_subplot(gs0[0,1])
ax010=fig.add_subplot(gs0[1,0])

#min = -2
#max = 1.2040868
min=None
max=None

w_rec = (Model() & {'model_id':  model_id}).fetch1('w_rec')
w_in = (Model() & {'model_id':  model_id}).fetch1('w_in')
w_out = (Model() & {'model_id':  model_id}).fetch1('w_out')
sns.heatmap(gaussian_filter(w_in,sigma=.5),ax=ax001,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(gaussian_filter(w_rec,sigma=.5),ax=ax000,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(gaussian_filter(w_out,sigma=.5),ax=ax010,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
#ax0.set_title(r'$W$')
#min = -3
#max = 1.2040868

# Panel B 
gs1 = gs[1].subgridspec(2, 2,height_ratios=[1,.25],width_ratios=[1,.25])
gs11 = gs1[0].subgridspec(2, 2,height_ratios=[1,.125],width_ratios=[1,.25],hspace=.1,wspace=.1)
ax100=fig.add_subplot(gs11[0,0])
ax101=fig.add_subplot(gs11[0,1])
ax110=fig.add_subplot(gs11[1,0])

df = pd.DataFrame((LCA() ).proj('model_id','w_rec','w_in','w_out','q').fetch())
w_rec = np.mean(lca_data['w_rec'])
w_in = np.mean(lca_data['w_in'])
w_out = np.mean(lca_data['w_out'])

w_rec = np.mean((LCA() & {'model_id':  model_id}).fetch('w_rec'))
w_in = np.mean((LCA() & {'model_id':  model_id}).fetch('w_in'))
w_out = np.mean((LCA() & {'model_id':  model_id}).fetch('w_out'))
sns.heatmap(w_in,ax=ax101,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(w_rec,ax=ax100,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(w_out,ax=ax110,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)

# Panel C 
q = np.mean(lca_data['q'])
sns.heatmap(gaussian_filter(q.T,sigma=.75),center=0,ax=ax2,xticklabels=False,yticklabels=False,cbar=False)


#Panel C: Change of basis
gs2 = gs[3].subgridspec(2, 2,height_ratios=[1,.25],width_ratios=[1,.25])
gs22 = gs2[0].subgridspec(2, 2,height_ratios=[1,.125],width_ratios=[1,.25],hspace=.1,wspace=.1)
ax200=fig.add_subplot(gs22[0,0])
ax201=fig.add_subplot(gs22[0,1])
ax210=fig.add_subplot(gs22[1,0])
w_rec = (Model() & {'model_id':  model_id}).fetch1('w_rec')
w_in = (Model() & {'model_id':  model_id}).fetch1('w_in')
w_out = (Model() & {'model_id': model_id}).fetch1('w_out')
w_ext = np.hstack((w_in,w_out.T))
w_ext = q @w_ext 

sns.heatmap(w_ext[:6,:],ax=ax201,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(q@w_rec@q.T,ax=ax200,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)
sns.heatmap(w_ext[6:,:],ax=ax210,center=0,vmin=min,vmax=max,cbar=False,xticklabels=False,yticklabels=False)




# Panel D: scatter plot
# w_in = (Model() & {'model_id':  model_id}).fetch1('w_in')
# w_out = (Model() & {'model_id': model_id}).fetch1('w_out')
# w_ext = np.hstack((w_in,w_out.T))

# w_ext = q @w_ext 
# a = np.concatenate(((q@w_rec@q.T).flatten(),w_ext.flatten()))
# w_rec = np.mean((LCA() & {'model_id':  model_id}).fetch('w_rec'))
# w_in = np.mean((LCA() & {'model_id':  model_id}).fetch('w_in'))
# w_out = np.mean((LCA() & {'model_id':  model_id}).fetch('w_out'))
# w_ext = np.hstack((w_in,w_out.T))
# b = np.concatenate((w_rec.flatten(),w_ext.flatten()))
# sns.scatterplot(x=a.flatten(),y=b.flatten(),s=5,alpha=.5,ax=ax3,color='black')
# from scipy.stats import linregress
# slope, intercept, r_value, p_value, std_err = linregress(a.flatten(),b.flatten())
# X_plot = np.linspace(np.min(a.flatten()),np.max(a.flatten()),100)
# Y_plot = slope*X_plot+intercept
# ax3.plot(X_plot, Y_plot, linewidth=1,color='black')
# r,p= scipy.stats.pearsonr(a,b)

# ax3.set_xlabel('Actual',fontsize=8,labelpad=0)
# ax3.set_ylabel('Predicted ',fontsize=8,labelpad=0)
# ax3.spines["top"].set_visible(False)
# ax3.spines["right"].set_visible(False)
# for label in (ax3.get_xticklabels() + ax3.get_yticklabels()):
#         label.set_fontsize(6)
# ax3.tick_params(left=False)
# ax3.tick_params(bottom=False)       
# ax3.tick_params(axis='both', which='major', pad=0)      
# ax3.text(0,-2.5,r'$r=0.91$', fontsize=6)
# ax3.text(-.1,-3.5,r'$y=1.51x - .03$', fontsize=6)
        
for ax in [ax0,ax1,ax2,ax3]:
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.tick_params(left=False)
    ax.tick_params(bottom=False)

ax000.set_title(r'$W_{rec}$',fontsize=9)
ax001.set_ylabel(r'$W_{in}$',fontsize=9,rotation=-90,labelpad=10)
ax010.set_xlabel(r'$W_{out}$',fontsize=9)
ax001.yaxis.set_label_position("right")
ax000.set_xlabel('  Excitatory              Inh',fontsize=6,labelpad=1)
ax000.xaxis.set_label_position("top")

ax102.set_title(r'$Q$', fontsize=9)
ax100.set_title(r'$\bar{W}_{rec}$',fontsize=9)
ax101.set_ylabel(r'$\bar{W}_{in}$',fontsize=9,rotation=-90,labelpad=12)
ax110.set_xlabel(r'$\bar{W}_{out}$',fontsize=9)
ax101.yaxis.set_label_position("right")
#ax100.set_ylabel('Choice   Sensory    Cue',fontsize=5,labelpad=0)
ax100.set_xlabel('Cue   Sensory  Choice',fontsize=5,labelpad=1)
ax100.xaxis.set_label_position("top")
ax102.set_xlabel('Latent',fontsize=6,labelpad=1)
ax102.xaxis.set_label_position("top")
ax102.set_ylabel('RNN unit',fontsize=6,labelpad=6,rotation=-90)
ax102.yaxis.set_label_position("right")

ax200.set_title(r'$Q^TW_{rec}Q$',fontsize=9)
ax201.set_ylabel(r'$QW_{in}$',fontsize=9,rotation=-90,labelpad=10)
ax210.set_xlabel(r'$W_{out}Q$',fontsize=9)
ax201.yaxis.set_label_position("right")
#ax200.set_ylabel('Choice   Sensory    Cue',fontsize=5,labelpad=0)
ax200.set_xlabel('Cue   Sensory  Choice',fontsize=5,labelpad=1)
ax200.xaxis.set_label_position("top")

ax2.set_xlabel('Latent neuron',fontsize=6)
ax2.set_ylabel('RNN neuron',fontsize=6)
ax2.set_title(r'$Q$',fontsize=9)
plt.savefig('/users/langdon/figures/circuitpaperfigures/results_1.png',dpi=1000, transparent=True)

# Figure: Circuit

In [ ]:
# Panel E (Circuit)
ax4=fig.add_subplot(gs[1,1:])
latent_data = (LCA() & {'lca_id': lca_id}).fetch1()
w_rec = latent_data['w_rec']
w_in = latent_data['w_in']
w_out = latent_data['w_out']
A=np.zeros((16, 16))
#latent_w_rec[np.abs(latent_w_rec)<1.5]=0
A[:8,:8]=w_rec
A[:8,8:14]=w_in
A[14:,:8]=w_out
A = .5*A
G = nx.from_numpy_matrix(A.T, create_using=nx.DiGraph)
#pos = nx.shell_layout(G,[[0,1,2,3,4,5,6,7],[8,9,10,11,12,13,14,15]],rotate=0)
pos = {0: [-1,0],
      1: [1,0],
      2: [-3,1],
      3: [-2,1],
      4: [2,1],
      5: [3,1],
      6: [-1,2],
      7: [1,2],
       
      8: [-1,-1],
      9: [1,-1],
      
      10: [-5,0],
      11: [-4,0],
      
      12: [4,0],
      13: [5,0],
      
      14: [-1,3],
      15: [1,3]}

node_size = [50,50,50,50,50,50,50,50,25,25,25,25,25,25,25,25]
nx.draw_networkx_nodes(G, pos,node_color=['lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray',
                                          'lightgray','lightgray','lightgray','lightgray'],
                       edgecolors='black',
                       linewidths=.25,
                       node_size=[50,50,50,50,50,50,50,50,25,25,25,25,25,25,25,25],
                       alpha=.75,
                      ax=ax4)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
nx.draw_networkx_edges(G, pos,edge_color=weights,width=weights,connectionstyle='Angle3',edge_cmap=plt.cm.coolwarm,arrowsize=10, alpha =1,ax=ax4);

In [33]:
plt.figure(figsize=(3,3))
latent_data = (LCA() & {'lca_id': lca_id}).fetch1()
w_rec = latent_data['w_rec']
w_in = latent_data['w_in']
w_out = latent_data['w_out']
A=np.zeros((16, 16))
#latent_w_rec[np.abs(latent_w_rec)<1.5]=0
A[:8,:8]=w_rec
A[:8,8:14]=w_in
A[14:,:8]=w_out
A = .5*A
G = nx.from_numpy_matrix(A.T, create_using=nx.DiGraph)
#pos = nx.shell_layout(G,[[0,1,2,3,4,5,6,7],[8,9,10,11,12,13,14,15]],rotate=0)
pos = {0: [-1,0],
      1: [1,0],
      2: [-3,1],
      3: [-2,1],
      4: [2,1],
      5: [3,1],
      6: [-1,2],
      7: [1,2],
       
      8: [-1,-1],
      9: [1,-1],
      
      10: [-5,0],
      11: [-4,0],
      
      12: [4,0],
      13: [5,0],
      
      14: [-1,3],
      15: [1,3]}

node_size = [100,100,100,100,100,100,100,100,25,25,25,25,25,25,25,25]
nx.draw_networkx_nodes(G, pos,node_color=['lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray',
                                          'lightgray','lightgray','lightgray','lightgray'],
                       edgecolors='black',
                       linewidths=.25,
                       node_size=node_size,
                       alpha=.75)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
nx.draw_networkx_edges(G, pos,edge_color=weights,width=weights,connectionstyle='Angle3',edge_cmap=plt.cm.coolwarm,arrowsize=10, alpha =1);
plt.savefig('/users/langdon/figures/circuitpaperfigures/circuit.png',dpi=1000, transparent=True)

# Figure: Dynamics

In [44]:
lca_id = "ck0O4hwm"
model_id = (LCA() & {'lca_id': lca_id}).fetch1('model_id')
fig = plt.figure(figsize=(10,3))
gs = gridspec.GridSpec(2,4, figure=fig,wspace=.5,hspace=.5)
# RNN connectivity

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax3 = fig.add_subplot(gs[2])
ax4 = fig.add_subplot(gs[3])

q = (LCA() & {'lca_id':lca_id}).fetch1('q')
q =  q[[0,2,4,6],:] - q[[1,3,5,7],:]
    
t = 3000
dt = .2 * 200
n_t = int(round(t / dt))
trial_events = {'n_t': int(round(n_t)),
                'cue_on': int(round(n_t * .1)),
                'cue_off': int(round(n_t * .33)),
                'stim_on': int(round(n_t * .4)),
                'stim_off': int(round(n_t)),
                'dec_on': int(round(n_t * .75)),
                'dec_off': int(round(n_t))}

# Load inputs and labels
inputs, labels, mask, conditions = generate_trials(**trial_events,
                                          n_trials=5,
                                          alpha=0.2,
                                          tau=200,
                                          sigma_in=.01,
                                          baseline=0.2,
                                          n_coh=6)


n = (Model() & {'model_id':model_id}).fetch1('n')
size = (Model() & {'model_id':model_id}).fetch1('connectivity')
rnn = RNNModule(connectivity=size, mask=mask, n=n)

rnn.recurrent_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_rec'))
rnn.input_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_in'))
rnn.output_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_out'))

dt=1
x, z_mask, z = rnn.forward(inputs)
x = x[:,::dt,:].detach().numpy()
z = z[:,::dt,:].detach().numpy()
time = np.tile(40 * dt * np.arange(x.shape[1]),x.shape[0])[:,None]
x = np.reshape(x,(-1,x.shape[2])) @ q.T
z = np.diff(np.reshape(z,(-1,z.shape[2])))

conditions = np.repeat(pd.DataFrame(conditions).values, repeats=n_t // dt, axis=0)

data = np.concatenate((conditions,time,x,z),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','time','context_x','motion_x','color_x', 'choice_x','z'])

df["correct_choice"] = df["correct_choice"].astype(float)
df["time"] = df["time"].astype(float)
df["context_x"] = df["context_x"].astype(float)
df["motion_x"] = df["motion_x"].astype(float)
df["color_x"] = df["color_x"].astype(float)
df["choice_x"] = df["choice_x"].astype(float)
df["motion_coh"] = df["motion_coh"].astype(float)
df["z"] = df["z"].astype(float)

 # Plotting
fontsize = 6
lw=1

s1=sns.lineplot(
data=df,
x="time",
y="context_x",
style='context',
color='gray',
legend=True,
ax = ax1,
    ci='sd',
    lw=lw
)

ax1.axvspan(40*trial_events['cue_on'], 40*trial_events['cue_off'], alpha=0.35, color='lightgray')
ax1.axvspan(40*trial_events['stim_on'], 40*trial_events['stim_off'], alpha=0.35, color='lightgray')
ax1.axvspan(40*trial_events['dec_on'], 40*trial_events['dec_off'], alpha=0.35, color='lightgray')
for label in (ax1.get_xticklabels() + ax1.get_yticklabels()):
    label.set_fontsize(fontsize)
ax1.set_xticklabels([])


ax1.set_ylabel("")
#ax1.tick_params(bottom=False)
#ax1.tick_params(left=False)
ax1.spines["top"].set_visible(False)
ax1.spines["right"].set_visible(False)
#plt.legend(loc='upper left')

#Motion 

palette = sns.color_palette("coolwarm", 6)
sns.lineplot(
data=df,
x="time",
y="motion_x",
hue='motion_coh',
style='context',
palette=palette,
legend=False,
    ax = ax2,
ci='sd',
    lw=lw
)
ax2.axvspan(40*trial_events['stim_on'], 40*trial_events['stim_off'], alpha=0.35, color='lightgray')


for label in (ax2.get_xticklabels() + ax2.get_yticklabels()):
    label.set_fontsize(fontsize)
ax2.set_xticklabels([])
ax2.set_ylabel("")
ax2.tick_params(bottom=False)
ax2.tick_params(left=False)
ax2.spines["top"].set_visible(False)
ax2.spines["right"].set_visible(False)

#plt.legend(loc='upper left')

# Color 

palette = sns.color_palette("coolwarm", 6)
sns.lineplot(
data=df,
x="time",
y="color_x",
hue = 'color_coh',
style='context',
palette=palette,
legend=False,
    ax = ax3,
    ci='sd',
    lw=lw
)
ax3.axvspan(40*trial_events['stim_on'], 40*trial_events['stim_off'], alpha=0.35, color='lightgray')
for label in (ax3.get_xticklabels() + ax3.get_yticklabels()):
    label.set_fontsize(fontsize)

ax3.set_ylabel("")
ax3.set_xticklabels([])
ax3.tick_params(bottom=False)
ax3.tick_params(left=False)
ax3.spines["top"].set_visible(False)
ax3.spines["right"].set_visible(False)

#plt.legend(loc='upper left')

# Choice

palette = sns.color_palette("coolwarm", 2)
s4=sns.lineplot(
data=df,
x="time",
y="choice_x",
hue = 'correct_choice',
style='context',
palette=palette,
legend='brief',
    ax = ax4,
    ci='sd',
    lw=lw
)
ax4.axvspan(40*trial_events['dec_on'], 40*trial_events['dec_off'], alpha=0.25, color='lightgray')
for label in (ax4.get_xticklabels() + ax4.get_yticklabels()):
    label.set_fontsize(fontsize)

ax4.set_ylabel("")
ax4.set_xticklabels([])
ax4.tick_params(bottom=False)
ax4.tick_params(left=False)
ax4.spines["top"].set_visible(False)
ax4.spines["right"].set_visible(False)


for ax in [ax1,ax2,ax3,ax4]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(fontsize)
    ax.set_xticklabels([])
    ax.set_ylabel("")
    ax.set_xlabel("")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

plt.setp(s1.get_legend().get_texts(), fontsize='6') 
plt.setp(s1.get_legend().get_title(), fontsize='6') 



#PLOT FITS AND 2D PROJECTIONS
t = 3000
dt = .2 * 200
n_t = int(round(t / dt))
trial_events = {'n_t': int(round(n_t)),
                'cue_on': int(round(n_t * .1)),
                'cue_off': int(round(n_t * .33)),
                'stim_on': int(round(n_t * .4)),
                'stim_off': int(round(n_t)),
                'dec_on': int(round(n_t * .75)),
                'dec_off': int(round(n_t))}

# Load inputs and labels
inputs, labels, mask, conditions = generate_trials(**trial_events,
                                          n_trials=5,
                                          alpha=0.2,
                                          tau=200,
                                          sigma_in=.01,
                                          baseline=0.2,
                                          n_coh=4)

n = (Model() & {'model_id':model_id}).fetch1('n')
size = (Model() & {'model_id':model_id}).fetch1('connectivity')
rnn = RNNModule(connectivity=size, mask=mask, n=n)

rnn.recurrent_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_rec'))
rnn.input_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_in'))
rnn.output_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_out'))
rnn_x, z_mask, z = rnn.forward(inputs)

dt=1
x = rnn_x[:,::dt,:].detach().numpy()
z = z[:,::dt,:].detach().numpy()
repeats= x.shape[1]
time = np.tile(40 * dt * np.arange(x.shape[1]),x.shape[0])[:,None]
q = (LCA() & {'lca_id':lca_id}).fetch1('q')
x = np.reshape(x,(-1,x.shape[2])) @ q.T
z = np.reshape(z,(-1,z.shape[2]))
conditions = np.repeat(pd.DataFrame(conditions).values, repeats=repeats, axis=0)

data = np.concatenate((conditions,time,x,z),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','time','context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'])
df["correct_choice"] = df["correct_choice"].astype(float)
df["time"] = df["time"].astype(float)
df["motion_coh"] = df["motion_coh"].astype(float)
df1 = df.groupby(['context','motion_coh','color_coh','correct_choice','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()


# Latent circuit activity
n = 8

rnn = RNNModule(connectivity='small', mask=mask, n=n,sigma_rec=.15)
rnn.recurrent_layer.weight.data = torch.tensor((LCA() & {'lca_id':lca_id}).fetch1('w_rec'))
rnn.input_layer.weight.data = torch.tensor((LCA() & {'lca_id':lca_id}).fetch1('w_in'))
rnn.output_layer.weight.data = torch.tensor((LCA() & {'lca_id':lca_id}).fetch1('w_out'))

x, z_mask, z = rnn.forward(inputs)
x = x[:,::dt,:].detach().numpy()
z = z[:,::dt,:].detach().numpy()
time = np.tile(40 * dt * np.arange(x.shape[1]),x.shape[0])[:,None]
x = np.reshape(x,(-1,x.shape[2])) 
z = np.reshape(z,(-1,z.shape[2]))
data = np.concatenate((conditions,time,x,z),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','time','context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'])

df["correct_choice"] = df["correct_choice"].astype(float)
df["time"] = df["time"].astype(float)
df["motion_coh"] = df["motion_coh"].astype(float)
df2 = df.groupby(['context','motion_coh','color_coh','correct_choice','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()

#fig = plt.figure(figsize=(18,3.5))

ax5 = fig.add_subplot(gs[4])
ax6 = fig.add_subplot(gs[5])
ax7 = fig.add_subplot(gs[6])
ax8 = fig.add_subplot(gs[7])

for label in (ax5.get_xticklabels() + ax5.get_yticklabels()):
    label.set_fontsize(fontsize)
for label in (ax6.get_xticklabels() + ax6.get_yticklabels()):
    label.set_fontsize(fontsize)
for label in (ax7.get_xticklabels() + ax7.get_yticklabels()):
    label.set_fontsize(fontsize)
for label in (ax8.get_xticklabels() + ax8.get_yticklabels()):
    label.set_fontsize(fontsize)


markersize=2
markevery=2
lw=1

# Context subspace
df= df1.groupby(['context','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()

cohs=np.linspace(-.2,.2,2)
ax5.plot(df[(df['context']=='motion') ]['context_x_1'],df[(df['context']=='motion')]['context_x_2'],color='gray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
ax5.plot(df[(df['context']=='color')]['context_x_1'],df[(df['context']=='color')]['context_x_2'],color='gray',lw=lw)

df= df2.groupby(['context','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()
ax5.plot(df[(df['context']=='motion') ]['context_x_1'],df[(df['context']=='motion')]['context_x_2'],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
ax5.plot(df[(df['context']=='color')]['context_x_1'],df[(df['context']=='color')]['context_x_2'],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)

# Motion subspace
df = df1.groupby(['context','motion_coh','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()
cohs=np.linspace(-.2,.2,4)
for i in range(4):
    ax6.plot((df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='gray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
    ax6.plot((df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='gray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
df = df2.groupby(['context','motion_coh','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()
for i in range(4):
    ax6.plot((df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
    ax6.plot((df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)

# Color subspace
df = df1.groupby(['context','color_coh','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()
cohs=np.linspace(-.2,.2,4)
for i in range(4):
    ax7.plot((df[(df['context']=='motion') & (df['color_coh']==cohs[i])]['color_x_1'])[30:],(df[(df['context']=='motion') & (df['color_coh']==cohs[i])]['color_x_2'])[30:],color='gray',marker='.',markersize=markersize,markevery=markevery,lw=lw)
    ax7.plot((df[(df['context']=='color') & (df['color_coh']==cohs[i])]['color_x_1'])[30:],(df[(df['context']=='color') & (df['color_coh']==cohs[i])]['color_x_2'])[30:],color='gray',marker='.',markersize=markersize,markevery=markevery,lw=lw)
df = df2.groupby(['context','color_coh','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()
for i in range(4):
    ax7.plot((df[(df['context']=='motion') & (df['color_coh']==cohs[i])]['color_x_1'])[30:],(df[(df['context']=='motion') & (df['color_coh']==cohs[i])]['color_x_2'])[30:],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
    ax7.plot((df[(df['context']=='color') & (df['color_coh']==cohs[i])]['color_x_1'])[30:],(df[(df['context']=='color') & (df['color_coh']==cohs[i])]['color_x_2'])[30:],lw=lw,color='lightgray',marker='.',markersize=markersize,markevery=markevery)

# Choice subspace
df = df1.groupby(['correct_choice','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()  
ax8.plot(df[(df['correct_choice']==1) ]['choice_x_1'],df[(df['correct_choice']==1)]['choice_x_2'],color='gray',marker='.',markersize=markersize,markevery=markevery,lw=lw)
ax8.plot(df[(df['correct_choice']==-1)]['choice_x_1'],df[(df['correct_choice']==-1)]['choice_x_2'],marker='.',markersize=markersize,markevery=markevery,color='gray',lw=lw)

df = df2.groupby(['correct_choice','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()  
ax8.plot(df[(df['correct_choice']==1) ]['choice_x_1'],df[(df['correct_choice']==1)]['choice_x_2'],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
ax8.plot(df[(df['correct_choice']==-1)]['choice_x_1'],df[(df['correct_choice']==-1)]['choice_x_2'],lw=lw,color='lightgray',marker='.',markersize=markersize,markevery=markevery)


for ax in [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8]:
    ax.tick_params(bottom=False)
    ax.tick_params(left=False)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.tick_params(axis='both', which='major', pad=1)
    
ax1.set_xlabel('Time', fontsize=8)
ax1.set_ylabel(r'$(Q_1-Q_0)^T\mathbf{x}$', fontsize=8, labelpad=-1)

ax2.set_xlabel('Time', fontsize=8)
ax2.set_ylabel(r'$(Q_3-Q_2)^T\mathbf{x}$', fontsize=8, labelpad=-1)

ax3.set_xlabel('Time', fontsize=8)
ax3.set_ylabel(r'$(Q_5-Q_4)^T\mathbf{x}$', fontsize=8, labelpad=-1)

ax4.set_xlabel('Time', fontsize=8)
ax4.set_ylabel(r'$(Q_7-Q_6)^T\mathbf{x}$', fontsize=8, labelpad=-1)

ax5.set_xlabel(r'$Q_0^T\mathbf{x}$', fontsize=8, labelpad=-1)
ax5.set_ylabel(r'$Q_1^T\mathbf{x}$', fontsize=8, labelpad=-1)

ax6.set_xlabel(r'$Q_2^T\mathbf{x}$', fontsize=8, labelpad=-1)
ax6.set_ylabel(r'$Q_3^T\mathbf{x}$', fontsize=8, labelpad=-1)

ax7.set_xlabel(r'$Q_4^T\mathbf{x}$', fontsize=8, labelpad=-1)
ax7.set_ylabel(r'$Q_5^T\mathbf{x}$', fontsize=8, labelpad=-1)

ax8.set_xlabel(r'$Q_6^T\mathbf{x}$', fontsize=8, labelpad=-1)
ax8.set_ylabel(r'$Q_7^T\mathbf{x}$', fontsize=8, labelpad=-1)

# Variance explained
rnn_x = rnn_x.detach().reshape(-1,150)
rnn_x = rnn_x- torch.mean(rnn_x,dim=0,keepdim=True)
q = (LCA() & {'lca_id':lca_id}).fetch1('q')
proj = q[[0,1],:]
r_context = np.trace(np.cov((rnn_x @ proj.T).T)) / np.trace(np.cov(rnn_x.T))
proj = q[[2,3],:]
r_motion = np.trace(np.cov((rnn_x @ proj.T).T)) / np.trace(np.cov(rnn_x.T))
proj = q[[4,5],:]
r_color = np.trace(np.cov((rnn_x @ proj.T).T)) / np.trace(np.cov(rnn_x.T))
proj = q[[6,7],:]
r_choice = np.trace(np.cov((rnn_x @ proj.T).T)) / np.trace(np.cov(rnn_x.T))
x=.5
y=.75
ax5.text(x,y,'r =  '+ '{:.1f}'.format(100*r_context) + ' % ',fontsize=7,transform = ax5.transAxes,color='gray')
ax6.text(x,y,'r =  '+ '{:.1f}'.format(100*r_motion) + ' % ',fontsize=7,transform = ax6.transAxes,color='gray')
ax7.text(x,y,'r =  '+ '{:.1f}'.format(100*r_color) + ' % ',fontsize=7,transform = ax7.transAxes,color='gray')
ax8.text(x,y,'r =  '+ '{:.1f}'.format(100*r_choice) + ' % ',fontsize=7,transform = ax8.transAxes,color='gray')
plt.savefig('/users/langdon/figures/circuitpaperfigures/dynamics.png',dpi=1000, transparent=True)

# Figure: perturbed psychometrics

In [2]:
lca_id = "8OqAUOwV"
model_id = (LCA() & {'lca_id': lca_id}).fetch1('model_id')
fig = plt.figure(figsize=(4,4.5))
plt.gcf().subplots_adjust(bottom=0.25)


gs = gridspec.GridSpec(2,2, figure=fig,wspace=.5,hspace=.75)

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])


query = Model() & {'model_id': model_id}
perturbation = Perturbation(tau=200,
                            sigma_rec=float(0.15),
                            n=query.fetch1('n'))
perturbation.input_layer.weight.data = torch.tensor(query.fetch1('w_in')).float()
perturbation.recurrent_layer.weight.data = torch.tensor(query.fetch1('w_rec')).float()
perturbation.output_layer.weight.data = torch.tensor(query.fetch1('w_out')).float()
perturbation.to(device=device)

# Define trial structure.
t = 3000
dt = 200 * .2
n_t = int(round(t / dt))
trial_events = {'n_t': int(round(n_t)),
                'cue_on': int(round(n_t * .1)),
                'cue_off': int(round(n_t * .33)),
                'stim_on': int(round(n_t * .4)),
                'stim_off': int(round(n_t)),
                'dec_on': int(round(n_t * .75)),
                'dec_off': int(round(n_t))}

# Generate dataset for simulation

inputs, labels, mask, conditions  = generate_trials(**trial_events,
                                            n_trials=25,
                                            alpha=float(0.2),
                                            tau=perturbation.tau,
                                            sigma_in=.01,
                                            baseline=0.2,
                                            n_coh=15)

q = (LCA() & {'lca_id': lca_id}).fetch1('q')


print('Plotting')

def pf(x, alpha, beta):
    return 1. / (1 + np.exp( -(x-alpha)/beta ))

par0 = scipy.array([0, 1.])
# Context perturbation
column=2
strength=.175
strengths=np.linspace(0,.3,3)
dash_params= [2,4,6]
for i in range(3):
    print(i)
    strength = strengths[i]
    p = q[None,column,:]-q[None,column+1,:]
    p = strength * p / np.linalg.norm(p, axis=1)
    p = torch.tensor(p).float().to(device=device)

    inputs = inputs.to(device)

    z, x = perturbation.forward(inputs,p, float(0.2))
    z = z.detach().cpu().numpy()

    data=(np.concatenate(( pd.DataFrame(conditions).values,np.maximum(np.sign(z[:,-1,0,None]-z[:,-1,1,None]),0)),1))
    df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','output'])

    motion_df = df[df['context']=='motion'].groupby(['motion_coh', 'color_coh'])['output'].apply(np.mean).reset_index()
    color_df = df[df['context']=='color'].groupby(['motion_coh', 'color_coh'])['output'].apply(np.mean).reset_index()

    motion_motion_df = motion_df.groupby(['motion_coh'])['output'].apply(np.mean).reset_index()
    motion_color_df = motion_df.groupby(['color_coh'])['output'].apply(np.mean).reset_index()
    
    par, mcov = curve_fit(pf, motion_motion_df['motion_coh'].values, motion_motion_df['output'].values, par0)
    cohs = np.linspace(-.2,.2,15)
    #ax0.plot(cohs, pf(cohs, par[0], par[1]),linestyle='--',dashes=dash_params[i],color='k',linewidth=1)
    ax0.plot(cohs, pf(cohs, par[0], par[1]),color='k',linestyle='--',linewidth=1,dashes=[dash_params[i],dash_params[i]])
    print(r'$\alpha = ' +  str(par[0]) + ', $\beta = ' + str([par[1]]))
    par, mcov = curve_fit(pf, motion_color_df['color_coh'].values, motion_color_df['output'].values, par0)
    cohs = np.linspace(-.2,.2,50)
    ax1.plot(cohs, pf(cohs, par[0], par[1]),linestyle='--',dashes=[dash_params[i],dash_params[i]],color='k',linewidth=1)



ax0.set_xlabel('Motion coherence',fontsize=8)
ax0.set_ylabel('Choice to right ratio',fontsize=8)
ax1.set_ylim(0,1)
ax1.set_xlabel('Color coherence',fontsize=8)
ax1.set_ylabel('')

for ax in [ax0,ax1]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(6)
    ax.tick_params(axis='both', which='major', pad=1)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

# TWO Dimensional Psycho
query = Model() & {'model_id': model_id}
perturbation = Perturbation(tau=200,
                            sigma_rec=float(0.15),
                            n=query.fetch1('n'))
perturbation.input_layer.weight.data = torch.tensor(query.fetch1('w_in')).float()
perturbation.recurrent_layer.weight.data = torch.tensor(query.fetch1('w_rec')).float()
perturbation.output_layer.weight.data = torch.tensor(query.fetch1('w_out')).float()
perturbation.to(device=device)

# Define trial structure.
t = 3000
dt = 200 * .2
n_t = int(round(t / dt))
trial_events = {'n_t': int(round(n_t)),
                'cue_on': int(round(n_t * .1)),
                'cue_off': int(round(n_t * .33)),
                'stim_on': int(round(n_t * .4)),
                'stim_off': int(round(n_t)),
                'dec_on': int(round(n_t * .75)),
                'dec_off': int(round(n_t))}

# Generate dataset for simulation

inputs, labels, mask, conditions  = generate_trials(**trial_events,
                                            n_trials=50,
                                            alpha=float(0.2),
                                            tau=perturbation.tau,
                                            sigma_in=.01,
                                            baseline=0.2,
                                            n_coh=6)

q = (LCA() & {'lca_id': lca_id}).fetch1('q')


#Motion perturbation
column=2
strength=.15
p = q[None,column,:]-q[None,column+1,:]
p = strength * p / np.linalg.norm(p, axis=1)
p = torch.tensor(p).float().to(device=device)

inputs = inputs.to(device)

z, x = perturbation.forward(inputs,p, float(0.2))
z = z.detach().cpu().numpy()

data = np.concatenate((pd.DataFrame(conditions).values,np.maximum(np.sign(z[:,-1,1,None]-z[:,-1,0,None]),0)),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','output'])

motion_df = df[df['context']=='motion'].groupby(['motion_coh', 'color_coh'])
color_df = df[df['context']=='color'].groupby(['motion_coh', 'color_coh'])

motion_df = ((motion_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')
color_df = ((color_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')


sns.heatmap(motion_df,center=0, ax = ax2, xticklabels=motion_df.columns.values.round(2),yticklabels=motion_df.index.values.round(2),linewidths=.1,cbar=True)
ax2.set_xlabel('Color coherence',fontsize=8)
ax2.set_ylabel("Motion coherence",fontsize=8)
ax2.set_title("Motion context", fontsize=8)

sns.heatmap(color_df,center=0, ax = ax3, xticklabels=motion_df.columns.values.round(2),yticklabels=False,cbar=True,linewidths=.1)
ax3.set_xlabel('Color coherence',fontsize=8)
ax3.set_ylabel("")
ax3.set_title("Color context", fontsize=8)

cbar = ax2.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)
cbar = ax3.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)
for ax in [ax2,ax3]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(6)
    ax.tick_params(axis='both', which='major', pad=1)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.tick_params(bottom=False)
    ax.tick_params(left=False)

plt.savefig('/users/langdon/figures/circuitpaperfigures/perturbations.png',dpi=1000, transparent=True)

Plotting
0
$\alpha = -0.005724131017647265, eta = [0.010430934763754923]
1
$\alpha = -0.05429162542502033, eta = [0.01011970187850205]
2
$\alpha = -0.10313067585894131, eta = [0.010645481389557065]


# Figure: Schematic

In [10]:
(Model() & 'n=100')

model_id unique model id,connectivity specify connectivity structure,n number of neurons,activation,lr learning rate,batch_size,patience,threshold,lambda_r firing rate regularization constant,lambda_o orthogonality regularization constant,lambda_w weight regularization,r2,valid_loss,train_loss,epochs,l2_ortho,l2_rate,l2_weight,l2_task,w_rec,w_in,w_out,embedding,train_loss_history,valid_loss_history
4UcXnUfj,large,100,ReLU(),0.010000,128,25,0.00010,0.010000,1.000000,0.000000,0.94008,0.00959785,0.00774316,95,6.77175e-07,0.0232346,0.0365566,0.00936483,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
8779VrSF,large,100,ReLU(),0.010000,128,25,0.00010,0.010000,1.000000,0.000000,0.965879,0.00547794,0.00609884,202,5.69741e-06,0.0139532,0.0519283,0.00533271,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
9TCkDNoa,large,100,ReLU(),0.010000,128,25,0.00010,0.010000,1.000000,0.000000,0.95209,0.00768561,0.00983352,129,1.08245e-06,0.0196779,0.042267,0.00748775,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
A8lbFuFo,large,100,ReLU(),0.010000,128,25,0.00010,0.010000,1.000000,0.000000,0.958275,0.00688354,0.00617063,115,0.000161363,0.020106,0.0397993,0.00652112,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
aXkeWGog,large,100,ReLU(),0.010000,128,25,0.00010,0.010000,1.000000,0.000000,0.95623,0.00708976,0.00636787,121,0.0,0.0248913,0.0406732,0.00684085,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
B4Yi1DLo,large,100,ReLU(),0.010000,128,25,0.00010,0.010000,1.000000,0.000000,0.903315,0.0152945,0.0114982,154,8.85991e-06,0.0174841,0.0448437,0.0151108,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
D0eVLfN8,large,100,ReLU(),0.010000,128,25,0.00010,0.010000,1.000000,0.000000,0.366386,0.0993759,0.0993369,40,2.01372e-07,0.0348698,0.027141,0.099027,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
D6iflgeW,large,100,ReLU(),0.010000,128,25,0.00010,0.010000,1.000000,0.000000,0.939914,0.00958898,0.00834201,92,3.0859e-07,0.0197842,0.0362978,0.00939083,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
eaD7CfhW,large,100,ReLU(),0.010000,128,25,0.00010,0.010000,1.000000,0.000000,0.964779,0.005693,0.0061561,99,3.95248e-07,0.0187984,0.0373893,0.00550462,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
ei0V524i,large,100,ReLU(),0.010000,128,25,0.00010,0.010000,1.000000,0.000000,0.956714,0.00697839,0.0178345,117,9.38855e-06,0.0203908,0.0416612,0.00676509,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=


In [18]:
LCA() & ( Model() & 'n=100').proj('model_id')

model_id unique model id,lca_id,alpha,sigma_rec,activation,lr,weight_decay,patience,threshold,n_trials,batch_size,max_epochs,epochs,r2,r2_x,r2_xqt,r2_z,valid_loss,train_loss,valid_loss_history,train_loss_history,w_rec,w_in,w_out,q,a,w_error,q_error
4UcXnUfj,bXAY09Jt,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,292,0.0586,0.0196,0.0475,0.0985,0.05860,0.05641,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
4UcXnUfj,wwz8jlIS,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,256,0.0572,0.0196,0.0460,0.0947,0.05723,0.05857,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
8779VrSF,b03JgjEw,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,315,0.0704,0.0433,0.0465,0.0739,0.07038,0.07524,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
8779VrSF,eBv4aIO0,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,212,0.0786,0.0438,0.0548,0.0956,0.07866,0.07599,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
8779VrSF,kChSPLrH,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,230,0.0752,0.0427,0.0510,0.0918,0.07520,0.07478,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
8779VrSF,PNd7xNdn,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,201,0.0751,0.0433,0.0521,0.0852,0.07512,0.07344,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
9TCkDNoa,BGwsXRrX,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,275,0.0503,0.0216,0.0406,0.0597,0.05035,0.05163,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
9TCkDNoa,btkQIQOO,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,332,0.0483,0.0208,0.0375,0.0655,0.04839,0.05063,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
9TCkDNoa,sJkzk6aJ,0.20,0.15,ReLU(),0.0200000,0.00000,50,0.0001000,64,128,10000,347,0.0506,0.0211,0.0411,0.0653,0.05064,0.05043,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan


In [48]:
model_id = "8779VrSF"
lca_id = "b03JgjEw"
fig = plt.figure(figsize=(5,5.5))

gs = gridspec.GridSpec(2,2, figure=fig)

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])


latent_data = (LCA() & {'lca_id': lca_id}).fetch1()
w_rec_bar = latent_data['w_rec']
w_in_bar = latent_data['w_in']
w_out_bar = latent_data['w_out']
Abar=np.zeros((16, 16))
Abar[:8,:8]=w_rec_bar
Abar[:8,8:14]=w_in_bar
Abar[14:,:8]=w_out_bar
Abar = .5*Abar

model_data = (Model() & {'model_id': model_id}).fetch1()
w_rec = model_data['w_rec']
w_in = model_data['w_in']
w_out = model_data['w_out']
n = model_data['n']
A=np.zeros((n+8, n+8))
A[:n,:n]=w_rec
A[:n,n:n+6]=w_in
A[n+6:,:n]=w_out
inh_percentile = np.percentile(A[A<0].flatten(),.25)
exc_percentile = np.percentile(A[A>0].flatten(),98)
A[np.logical_and(A>inh_percentile, A<exc_percentile)]=0




# Panel A: highlight node in small circuit
axis=1
G = nx.from_numpy_matrix(Abar.T, create_using=nx.DiGraph)
pos = nx.shell_layout(G,[[0,1,2,3,4,5,6,7],[8,9,10,11,12,13,14,15]],rotate=0)
node_size = [50,50,50,50,50,50,50,50,25,25,25,25,25,25,25,25]
node_size[axis]=200
nx.draw_networkx_nodes(G, pos,node_color=['lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray',
                                          'lightgray','lightgray','lightgray','lightgray'],
                       edgecolors='black',
                       linewidths=.25,
                       node_size=node_size,
                       alpha=.75,
                      ax=ax0)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
nx.draw_networkx_edges(G, pos,edge_color='lightgray',width=weights,connectionstyle='Angle3',edge_cmap=plt.cm.coolwarm,arrowsize=10, alpha =1,ax=ax0);

# Panel B: Highlight column of Q in RNN
G = nx.from_numpy_matrix(A.T, create_using=nx.DiGraph)
pos = nx.shell_layout(G,[np.arange(n),np.arange(n,n+8)],rotate=0)
q = latent_data['q']
node_color = np.concatenate((q[axis,:],np.zeros((8))))
nx.draw_networkx_nodes(G, pos,cmap=plt.get_cmap('coolwarm'), node_color=node_color,
                       edgecolors='black',
                       linewidths=.25,
                       node_size=np.concatenate((500*np.abs(q[axis,:]),25*np.ones((8)))),
                       alpha=.75,
                      ax=ax1)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items());
nx.draw_networkx_edges(G, pos,edge_color='lightgray',width=weights,connectionstyle='Angle3',edge_cmap=plt.cm.coolwarm,arrowsize=10, alpha = 1,ax=ax1)
nx.draw_networkx(G.subgraph(np.arange(n,n+8)), pos=pos, node_color='lightgray',ax=ax1,alpha=1,edgecolors='black',linewidths=.25,node_size=25,with_labels=False)


# Panel C: Highlight egdges in RNN
G = nx.from_numpy_matrix(Abar.T, create_using=nx.DiGraph)
pos = nx.shell_layout(G,[[0,1,2,3,4,5,6,7],[8,9,10,11,12,13,14,15]],rotate=0)
nx.draw_networkx_nodes(G, pos,node_color=['lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray',
                                          'lightgray','lightgray','lightgray','lightgray'],
                       edgecolors='black',
                       linewidths=.25,
                       node_size=node_size,
                       alpha=.75,
                      ax=ax2)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
nx.draw_networkx_edges(G, pos,edge_color='lightgray',width=weights,connectionstyle='Angle3',arrowsize=10, alpha = 1,ax=ax2);

#highlights
Abarhighlight=np.zeros((16, 16))
Abarhighlight[:8,axis]=w_rec_bar[:,axis]
Abarhighlight=Abarhighlight*.5
G = nx.from_numpy_matrix(Abarhighlight.T, create_using=nx.DiGraph)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
nx.draw_networkx_edges(G, pos,edge_color=weights,width=weights,connectionstyle='Angle3',edge_cmap=plt.cm.coolwarm,arrowsize=10, alpha =1,ax=ax2);



# Panel D: Highlight edges in RNN

A[:n,:n]=w_rec * q[0,:]
A[:n,n:n+6]=w_in 
A[n+6:,:n]=w_out
inh_percentile = np.percentile(A[A<0].flatten(),.25)
exc_percentile = np.percentile(A[A>0].flatten(),99)
A[np.logical_and(A>inh_percentile, A<exc_percentile)]=0


G = nx.from_numpy_matrix(A.T, create_using=nx.DiGraph)
pos = nx.shell_layout(G,[np.arange(n),np.arange(n,n+8)],rotate=0)


q = latent_data['q']
node_color = np.concatenate((q[axis,:],np.zeros((8))))
#node_color = node_color - np.mean(node_color)
node_color[-8:]=np.mean(node_color)
nx.draw_networkx_nodes(G, pos,cmap=plt.get_cmap('coolwarm'), node_color='lightgray',
                       edgecolors='black',
                       linewidths=.25,
                       node_size=np.concatenate((500*np.abs(q[axis,:]),25*np.ones((8)))),
                       alpha=.75,
                      ax=ax3)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items());
nx.draw_networkx_edges(G, pos,edge_color='lightgray',width=weights,connectionstyle='Angle3',edge_cmap=plt.cm.coolwarm,arrowsize=10, alpha = 1,ax=ax3)

#highlights
A=np.zeros((n+8, n+8))

A[:n,:n]=w_rec * q[0,:]
inh_percentile = np.percentile(A[A<0].flatten(),.25)
exc_percentile = np.percentile(A[A>0].flatten(),98)
A[np.logical_and(A>inh_percentile, A<exc_percentile)]=0
G = nx.from_numpy_matrix(A.T, create_using=nx.DiGraph)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
nx.draw_networkx_edges(G, pos,edge_color=weights,width=weights,connectionstyle='Angle3',edge_cmap=plt.cm.coolwarm,arrowsize=10, alpha =1,ax=ax3);






# #  Panel B
# model_data = (Model() & {'model_id': model_id}).fetch1()
# w_rec = model_data['w_rec']
# w_in = model_data['w_in']
# w_out = model_data['w_out']

# A=np.zeros((n+8, n+8))

# A[:n,:n]=w_rec
# A[:n,n:n+6]=w_in
# A[n+6:,:n]=w_out
# inh_percentile = np.percentile(A[A<0].flatten(),.25)
# exc_percentile = np.percentile(A[A>0].flatten(),98)
# A[np.logical_and(A>inh_percentile, A<exc_percentile)]=0


# G = nx.from_numpy_matrix(A.T, create_using=nx.DiGraph)
# pos = nx.shell_layout(G,[np.arange(n),np.arange(n,n+8)],rotate=0)


# q = latent_data['q']

# node_color = np.concatenate((q[axis,:],np.zeros((8))))

# nx.draw_networkx_nodes(G, pos,cmap=plt.get_cmap('coolwarm'), node_color=node_color,
#                        edgecolors='black',
#                        linewidths=.25,
#                        node_size=np.concatenate((500*np.abs(q[axis,:]),25*np.ones((8)))),
#                        alpha=.75,
#                       ax=ax1)
# edges,weights = zip(*nx.get_edge_attributes(G,'weight').items());
# nx.draw_networkx_edges(G, pos,edge_color='lightgray',width=weights,connectionstyle='Angle3',edge_cmap=plt.cm.coolwarm,arrowsize=10, alpha = 1,ax=ax1)


# nx.draw_networkx(G.subgraph(np.arange(n,n+8)), pos=pos, node_color='lightgray',ax=ax1,alpha=1,edgecolors='black',linewidths=.25,node_size=25,with_labels=False)

# # THIRD ROW

# # Panel C: Highlight outgoing edges from node 0
# latent_data = (LCA() & {'lca_id': lca_id}).fetch1()
# w_rec = latent_data['w_rec']
# w_in = latent_data['w_in']
# w_out = latent_data['w_out']
# A=np.zeros((16, 16))
# A[:8,:8]=w_rec
# A[:8,8:14]=w_in
# A[14:,:8]=w_out
# A=A*.5
# G = nx.from_numpy_matrix(A.T, create_using=nx.DiGraph)poly
# pos = nx.shell_layout(G,[[0,1,2,3,4,5,6,7],[8,9,10,11,12,13,14,15]],rotate=0)


# nx.draw_networkx_nodes(G, pos,node_color=['lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray','lightgray',
#                                           'lightgray','lightgray','lightgray','lightgray'],
#                        edgecolors='black',
#                        linewidths=.25,
#                        node_size=node_size,
#                        alpha=.75,
#                       ax=ax2)
# edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
# nx.draw_networkx_edges(G, pos,edge_color='lightgray',width=weights,connectionstyle='Angle3',arrowsize=10, alpha = 1,ax=ax2);

# #highlights
# A=np.zeros((16, 16))
# A[:8,axis]=w_rec[:,axis]
# A=A*.5
# G = nx.from_numpy_matrix(A.T, create_using=nx.DiGraph)
# edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
# nx.draw_networkx_edges(G, pos,edge_color=weights,width=weights,connectionstyle='Angle3',edge_cmap=plt.cm.coolwarm,arrowsize=10, alpha =1,ax=ax2);


# # Panel D
# model_data = (Model() & {'model_id': model_id}).fetch1()
# w_rec = model_data['w_rec']
# w_in = model_data['w_in']
# w_out = model_data['w_out']

# A=np.zeros((n+8, n+8))

# A[:n,:n]=w_rec * q[0,:]
# A[:n,n:n+6]=w_in 
# A[n+6:,:n]=w_out
# inh_percentile = np.percentile(A[A<0].flatten(),.25)
# exc_percentile = np.percentile(A[A>0].flatten(),99)
# A[np.logical_and(A>inh_percentile, A<exc_percentile)]=0


# G = nx.from_numpy_matrix(A.T, create_using=nx.DiGraph)
# pos = nx.shell_layout(G,[np.arange(n),np.arange(n,n+8)],rotate=0)


# q = latent_data['q']
# node_color = np.concatenate((q[axis,:],np.zeros((8))))
# #node_color = node_color - np.mean(node_color)
# node_color[-8:]=np.mean(node_color)
# nx.draw_networkx_nodes(G, pos,cmap=plt.get_cmap('coolwarm'), node_color='lightgray',
#                        edgecolors='black',
#                        linewidths=.25,
#                        node_size=np.concatenate((500*np.abs(q[axis,:]),25*np.ones((8)))),
#                        alpha=.75,
#                       ax=ax3)
# edges,weights = zip(*nx.get_edge_attributes(G,'weight').items());
# nx.draw_networkx_edges(G, pos,edge_color='lightgray',width=weights,connectionstyle='Angle3',edge_cmap=plt.cm.coolwarm,arrowsize=10, alpha = 1,ax=ax3)

# #highlights
# A=np.zeros((n+8, n+8))

# A[:n,:n]=w_rec * q[0,:]
# inh_percentile = np.percentile(A[A<0].flatten(),.25)
# exc_percentile = np.percentile(A[A>0].flatten(),98)
# A[np.logical_and(A>inh_percentile, A<exc_percentile)]=0
# G = nx.from_numpy_matrix(A.T, create_using=nx.DiGraph)
# edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
# nx.draw_networkx_edges(G, pos,edge_color=weights,width=weights,connectionstyle='Angle3',edge_cmap=plt.cm.coolwarm,arrowsize=10, alpha =1,ax=ax3);


for ax in [ax0,ax1,ax2,ax3]:
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.tick_params(bottom=False)
    ax.tick_params(left=False)
    ax.tick_params(top=False)
    ax.tick_params(right=False)
    ax.axis('off')
plt.savefig('/users/langdon/figures/circuitpaperfigures/schematic.png',dpi=1000, transparent=True)




# Lesioning

In [139]:
lca_id = "8OqAUOwV"
model_id = (LCA() & {'lca_id': lca_id}).fetch1('model_id')
fig = plt.figure(figsize=(4,4.5))
plt.gcf().subplots_adjust(bottom=0.25)

gs = gridspec.GridSpec(2,2, figure=fig,wspace=.5,hspace=.75)

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])


query = Model() & {'model_id': model_id}
n = (Model() & {'model_id':model_id}).fetch1('n')
size = (Model() & {'model_id':model_id}).fetch1('connectivity')
rnn = RNNModule(connectivity=size, mask=mask, n=n)
q = (LCA() & {'lca_id': lca_id}).fetch1('q').T
p = 2*q[:,4,None]@q[:,0,None].T


rnn.recurrent_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_rec')+p)
rnn.input_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_in'))
rnn.output_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_out'))

# Define trial structure.
t = 3000
dt = 200 * .2
n_t = int(round(t / dt))
trial_events = {'n_t': int(round(n_t)),
                'cue_on': int(round(n_t * .1)),
                'cue_off': int(round(n_t * .33)),
                'stim_on': int(round(n_t * .4)),
                'stim_off': int(round(n_t)),
                'dec_on': int(round(n_t * .75)),
                'dec_off': int(round(n_t))}

# Generate dataset for simulation

inputs, labels, mask, conditions  = generate_trials(**trial_events,
                                            n_trials=25,
                                            alpha=float(0.2),
                                            tau=rnn.tau,
                                            sigma_in=.01,
                                            baseline=0.2,
                                            n_coh=7)


print('Plotting')

def pf(x, alpha, beta):
    return 1. / (1 + np.exp( -(x-alpha)/beta ))

par0 = scipy.array([0, 1.])
# Context perturbation

dash_params= [2,4,6]
for i in range(1):


    inputs = inputs.to(device)

    x, _, z = rnn.forward(inputs)
    z = z.detach().cpu().numpy()

    data=(np.concatenate(( pd.DataFrame(conditions).values,np.maximum(np.sign(z[:,-1,0,None]-z[:,-1,1,None]),0)),1))
    df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','output'])

    motion_df = df[df['context']=='motion'].groupby(['motion_coh', 'color_coh'])['output'].apply(np.mean).reset_index()
    color_df = df[df['context']=='color'].groupby(['motion_coh', 'color_coh'])['output'].apply(np.mean).reset_index()

    motion_motion_df = motion_df.groupby(['motion_coh'])['output'].apply(np.mean).reset_index()
    motion_color_df = motion_df.groupby(['color_coh'])['output'].apply(np.mean).reset_index()
    
    par, mcov = curve_fit(pf, motion_motion_df['motion_coh'].values, motion_motion_df['output'].values, par0)
    cohs = np.linspace(-.2,.2,15)
    #ax0.plot(cohs, pf(cohs, par[0], par[1]),linestyle='--',dashes=dash_params[i],color='k',linewidth=1)
    ax0.plot(cohs, pf(cohs, par[0], par[1]),color='k',linestyle='--',linewidth=1,dashes=[dash_params[i],dash_params[i]])
    print(r'$\alpha = ' +  str(par[0]) + ', $\beta = ' + str([par[1]]))
    par, mcov = curve_fit(pf, motion_color_df['color_coh'].values, motion_color_df['output'].values, par0)
    cohs = np.linspace(-.2,.2,50)
    ax1.plot(cohs, pf(cohs, par[0], par[1]),linestyle='--',dashes=[dash_params[i],dash_params[i]],color='k',linewidth=1)



ax0.set_xlabel('Motion coherence',fontsize=8)
ax0.set_ylabel('Choice to right ratio',fontsize=8)
ax1.set_ylim(0,1)
ax1.set_xlabel('Color coherence',fontsize=8)
ax1.set_ylabel('')

for ax in [ax0,ax1]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(6)
    ax.tick_params(axis='both', which='major', pad=1)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

# TWO Dimensional Psycho


inputs = inputs.to(device)

x, _, z = rnn.forward(inputs) 
z = z.detach().cpu().numpy()

data = np.concatenate((pd.DataFrame(conditions).values,np.maximum(np.sign(z[:,-1,1,None]-z[:,-1,0,None]),0)),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','output'])

motion_df = df[df['context']=='motion'].groupby(['motion_coh', 'color_coh'])
color_df = df[df['context']=='color'].groupby(['motion_coh', 'color_coh'])

motion_df = ((motion_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')
color_df = ((color_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')


sns.heatmap(motion_df,center=0, ax = ax2, xticklabels=motion_df.columns.values.round(2),yticklabels=motion_df.index.values.round(2),linewidths=.1,cbar=True)
ax2.set_xlabel('Color coherence',fontsize=8)
ax2.set_ylabel("Motion coherence",fontsize=8)
ax2.set_title("Motion context", fontsize=8)

sns.heatmap(color_df,center=0, ax = ax3, xticklabels=motion_df.columns.values.round(2),yticklabels=False,cbar=True,linewidths=.1)
ax3.set_xlabel('Color coherence',fontsize=8)
ax3.set_ylabel("")
ax3.set_title("Color context", fontsize=8)

cbar = ax2.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)
cbar = ax3.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)
for ax in [ax2,ax3]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(6)
    ax.tick_params(axis='both', which='major', pad=1)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.tick_params(bottom=False)
    ax.tick_params(left=False)

plt.savefig('/users/langdon/figures/circuitpaperfigures/perturbations.png',dpi=1000, transparent=True)

TypeError: generate_trials() got an unexpected keyword argument 'cue_on'

In [62]:
q = (LCA() & {'lca_id': lca_id}).fetch1('q').T
p = -10*q[:,6,None]@q[:,2,None].T

In [63]:
plt.figure()
sns.heatmap(p,center=0)

<AxesSubplot:>

In [64]:
p = -10*q[:,4,None]@q[:,0,None].T
plt.figure()
sns.heatmap(p,center=0)

<AxesSubplot:>

# Figure: Lesions

In [2]:
from scipy.ndimage import gaussian_filter


lca_id = "8OqAUOwV"
model_id = (LCA() & {'lca_id': lca_id}).fetch1('model_id')
fig = plt.figure(figsize=(8,3.5))
plt.gcf().subplots_adjust(bottom=0.25)

gs = gridspec.GridSpec(2,4, figure=fig,wspace=.5,hspace=.75)

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])

ax4 = fig.add_subplot(gs[4])
ax5 = fig.add_subplot(gs[5])
ax6 = fig.add_subplot(gs[6])
ax7 = fig.add_subplot(gs[7])

# Define trial structure.


# Panel A: lesion w_bar_6,2
w_bar = (LCA() & {'lca_id': lca_id}).fetch1('w_rec')
delta_w_bar = np.zeros_like(w_bar)
delta_w_bar[7,3]=w_bar[7,3]
#sns.heatmap(w_bar,center=0,alpha=.25,ax=ax0)
sns.heatmap(delta_w_bar,center=0,alpha=1,ax=ax0)

# Panel B: circuit psychometrics
inputs, labels, mask, conditions  = generate_trials(
                                            n_trials=25,
                                            alpha=float(0.2),
                                            tau=200,
                                            sigma_in=.01,
                                            baseline=0.2,
                                            n_coh=9)

rnn = RNNModule(connectivity='small', mask=mask, n=8)
w_rec = np.mean((LCA() & {'model_id':  model_id}).fetch('w_rec'))
w_in = np.mean((LCA() & {'model_id':  model_id}).fetch('w_in'))
w_out = np.mean((LCA() & {'model_id':  model_id}).fetch('w_out'))
rnn.recurrent_layer.weight.data = torch.tensor(w_rec)
p = torch.zeros((8,8))
p[7,3]=-2.25
rnn.recurrent_layer.weight.data = torch.tensor(w_rec)+p
rnn.input_layer.weight.data = torch.tensor(w_in)
rnn.output_layer.weight.data = torch.tensor(w_out)

x, _, z = rnn.forward(inputs) 
z = z.detach().cpu().numpy()
data = np.concatenate((pd.DataFrame(conditions).values,np.maximum(np.sign(z[:,-1,0,None]-z[:,-1,1,None]),0)),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','output'])
motion_df = df[df['context']=='motion'].groupby(['motion_coh', 'color_coh'])
motion_df = ((motion_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')
sns.heatmap(motion_df,center=0, ax = ax1, xticklabels=motion_df.columns.values.round(2),yticklabels=motion_df.index.values.round(2),linewidths=.1,cbar=True)


# Panel E
q = (LCA() & {'lca_id': lca_id}).fetch1('q').T
p_1 = -1*w_bar[7,3]*q[:,7,None]@q[:,3,None].T  
sns.heatmap(gaussian_filter(p_1, sigma=2),center=0,ax=ax4)


# Panel F
rnn = RNNModule(connectivity='large', mask=mask, n=150)
rnn.recurrent_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_rec') + p_1)
rnn.input_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_in'))
rnn.output_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_out'))
x, _, z = rnn.forward(inputs) 
z = z.detach().cpu().numpy()
data = np.concatenate((pd.DataFrame(conditions).values,np.maximum(np.sign(z[:,-1,0,None]-z[:,-1,1,None]),0)),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','output'])
motion_df = df[df['context']=='motion'].groupby(['motion_coh', 'color_coh'])
motion_df = ((motion_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')
sns.heatmap(motion_df,center=0, ax = ax5, xticklabels=motion_df.columns.values.round(2),yticklabels=motion_df.index.values.round(2),linewidths=.1,cbar=True)


# Panel C:
w_bar = (LCA() & {'lca_id': lca_id}).fetch1('w_rec')
delta_w_bar = np.zeros_like(w_bar)
delta_w_bar[4,0]=w_bar[4,0]

sns.heatmap(delta_w_bar,center=0,alpha=1,ax=ax2)


# Panel D: circuit psychometrics
inputs, labels, mask, conditions  = generate_trials(
                                            n_trials=25,
                                            alpha=float(0.2),
                                            tau=200,
                                            sigma_in=.01,
                                            baseline=0.2,
                                            n_coh=9)

rnn = RNNModule(connectivity='small', mask=mask, n=8)
w_rec = np.mean((LCA() & {'model_id':  model_id}).fetch('w_rec'))
w_in = np.mean((LCA() & {'model_id':  model_id}).fetch('w_in'))
w_out = np.mean((LCA() & {'model_id':  model_id}).fetch('w_out'))
rnn.recurrent_layer.weight.data = torch.tensor(w_rec)
rnn.recurrent_layer.weight.data[4,0]=0
rnn.input_layer.weight.data = torch.tensor(w_in)
rnn.output_layer.weight.data = torch.tensor(w_out)

x, _, z = rnn.forward(inputs) 
z = z.detach().cpu().numpy()
data = np.concatenate((pd.DataFrame(conditions).values,np.maximum(np.sign(z[:,-1,0,None]-z[:,-1,1,None]),0)),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','output'])
motion_df = df[df['context']=='motion'].groupby(['motion_coh', 'color_coh'])
motion_df = ((motion_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')
sns.heatmap(motion_df,center=0, ax = ax3, xticklabels=motion_df.columns.values.round(2),yticklabels=motion_df.index.values.round(2),linewidths=.1,cbar=True)


# Panel G
q = (LCA() & {'lca_id': lca_id}).fetch1('q').T
p_1 = -.6*w_bar[4,0]*q[:,4,None]@q[:,0,None].T  
sns.heatmap(gaussian_filter(p_1, sigma=2),center=0,ax=ax6)

# Panel H
rnn = RNNModule(connectivity='large', mask=mask, n=150)
rnn.recurrent_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_rec') + p_1)
rnn.input_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_in'))
rnn.output_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_out'))
x, _, z = rnn.forward(inputs) 
z = z.detach().cpu().numpy()
data = np.concatenate((pd.DataFrame(conditions).values,np.maximum(np.sign(z[:,-1,0,None]-z[:,-1,1,None]),0)),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','output'])
motion_df = df[df['context']=='motion'].groupby(['motion_coh', 'color_coh'])
motion_df = ((motion_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')
sns.heatmap(motion_df,center=0, ax = ax7, xticklabels=motion_df.columns.values.round(2),yticklabels=motion_df.index.values.round(2),linewidths=.1,cbar=True)


ax0.set_xlabel("Latent neuron",fontsize=6,labelpad=-2)
ax0.set_ylabel("Latent neuron",fontsize=6,labelpad=-2)
ax0.set_title(r'$\bar{W}_{7,3}$', fontsize=8)

ax1.set_xlabel('Color coherence',fontsize=6,labelpad=-2)
ax1.set_ylabel("Motion coherence",fontsize=6,labelpad=-2)
ax1.set_title("Motion context", fontsize=8)


ax2.set_xlabel("Latent neuron",fontsize=6,labelpad=-2)
ax2.set_ylabel("Latent neuron",fontsize=6,labelpad=-2)
ax2.set_title(r'$\bar{W}_{4,0}$', fontsize=8)

ax3.set_xlabel('Color coherence',fontsize=6,labelpad=-2)
ax3.set_ylabel("Motion coherence",fontsize=6,labelpad=-2)
ax3.set_title("Motion context", fontsize=8)

ax4.set_xlabel("RNN neuron",fontsize=6,labelpad=-2)
ax4.set_ylabel("RNN neuron",fontsize=6,labelpad=-2)
ax4.set_title(r'$Q_7\otimes Q_3$', fontsize=8)

ax5.set_xlabel('Color coherence',fontsize=6,labelpad=-2)
ax5.set_ylabel("Motion coherence",fontsize=6,labelpad=-2)
ax5.set_title("Motion context", fontsize=8)

ax6.set_xlabel("RNN neuron",fontsize=6,labelpad=-2)
ax6.set_ylabel("RNN neuron",fontsize=6,labelpad=-2)
ax6.set_title(r'$Q_4\otimes Q_0$', fontsize=8)

ax7.set_xlabel('Color coherence',fontsize=6,labelpad=-2)
ax7.set_ylabel("Motion coherence",fontsize=6,labelpad=-2)
ax7.set_title("Motion context", fontsize=8)




# query = Model() & {'model_id': model_id}
# n = (Model() & {'model_id':model_id}).fetch1('n')
# size = (Model() & {'model_id':model_id}).fetch1('connectivity')
# rnn = RNNModule(connectivity=size, mask=mask, n=n)
# q = (LCA() & {'lca_id': lca_id}).fetch1('q').T
# p_1 = -5*q[:,6,None]@q[:,2,None].T
# p_2 = 2*q[:,4,None]@q[:,0,None].T

# sns.heatmap(gaussian_filter(p_1, sigma=2),center=0,ax=ax0)
# sns.heatmap(gaussian_filter(p_2, sigma=2),center=0,ax=ax1)

# # Define trial structure.
# t = 3000
# dt = 200 * .2
# n_t = int(round(t / dt))
# trial_events = {'n_t': int(round(n_t)),
#                 'cue_on': int(round(n_t * .1)),
#                 'cue_off': int(round(n_t * .33)),
#                 'stim_on': int(round(n_t * .4)),
#                 'stim_off': int(round(n_t)),
#                 'dec_on': int(round(n_t * .75)),
#                 'dec_off': int(round(n_t))}

# # Generate dataset for simulation

# inputs, labels, mask, conditions  = generate_trials(**trial_events,
#                                             n_trials=25,
#                                             alpha=float(0.2),
#                                             tau=rnn.tau,
#                                             sigma_in=.01,
#                                             baseline=0.2,
#                                             n_coh=9)


# # Perturbation 1
# rnn.recurrent_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_rec')+p_1)
# rnn.input_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_in'))
# rnn.output_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_out'))

# inputs = inputs.to(device)

# x, _, z = rnn.forward(inputs) 
# z = z.detach().cpu().numpy()

# data = np.concatenate((pd.DataFrame(conditions).values,np.maximum(np.sign(z[:,-1,1,None]-z[:,-1,0,None]),0)),1)
# df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','output'])

# motion_df = df[df['context']=='motion'].groupby(['motion_coh', 'color_coh'])
# #color_df = df[df['context']=='color'].groupby(['motion_coh', 'color_coh'])

# motion_df = ((motion_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')
# #color_df = ((color_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')


# sns.heatmap(motion_df,center=0, ax = ax2, xticklabels=motion_df.columns.values.round(2),yticklabels=motion_df.index.values.round(2),linewidths=.1,cbar=True)



# # Perturbation 2

# rnn.recurrent_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_rec')+p_2)
# rnn.input_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_in'))
# rnn.output_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_out'))

# inputs = inputs.to(device)

# x, _, z = rnn.forward(inputs) 
# z = z.detach().cpu().numpy()

# data = np.concatenate((pd.DataFrame(conditions).values,np.maximum(np.sign(z[:,-1,1,None]-z[:,-1,0,None]),0)),1)
# df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','output'])

# motion_df = df[df['context']=='motion'].groupby(['motion_coh', 'color_coh'])
# #color_df = df[df['context']=='color'].groupby(['motion_coh', 'color_coh'])

# motion_df = ((motion_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')
# #color_df = ((color_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')

# sns.heatmap(motion_df,center=0, ax = ax3, xticklabels=motion_df.columns.values.round(2),yticklabels=False,cbar=True,linewidths=.1)



# cbar.ax.tick_params(labelsize=6)
# for ax in [ax2,ax3]:
#     for label in (ax.get_xticklabels() + ax.get_yticklabels()):
#         label.set_fontsize(6)
#     ax.tick_params(axis='both', which='major', pad=1)
#     ax.spines["top"].set_visible(False)
#     ax.spines["right"].set_visible(False)
#     ax.spines["bottom"].set_visible(False)
#     ax.spines["left"].set_visible(False)
#     ax.tick_params(bottom=False)
#     ax.tick_params(left=False)
    

# ax0.set_xlabel("")
# ax0.set_ylabel("")
# #ax1.set_ylim(0,1)
# ax1.set_xlabel("")
# ax1.set_ylabel("")
# ax0.set_title(r'$Q_6\otimes Q_2$',fontsize=10)
# ax1.set_title(r'$Q_4\otimes Q_0$',fontsize=10)

for ax in [ax0,ax1,ax2,ax3,ax4,ax5,ax6,ax7]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(6)
    ax.tick_params(axis='both', which='major', pad=1)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.tick_params(axis='both', which='major', pad=1)
    ax.tick_params(bottom=False)
    ax.tick_params(left=False)
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=5)
    
# ax2.set_xlabel('Color coherence',fontsize=8,labelpad=-1)
# ax2.set_ylabel("Motion coherence",fontsize=8,labelpad=-1)
# ax2.set_title("Motion context", fontsize=8)
# ax3.set_xlabel('Color coherence',fontsize=8,labelpad=-1)
# ax3.set_ylabel("")
# ax3.set_title("Motion context", fontsize=8)
plt.savefig('/users/langdon/figures/circuitpaperfigures/lesions.png',dpi=1000, transparent=True)

In [91]:
plt.figure()
sns.heatmap(gaussian_filter(p_2, sigma=2),center=0)

<AxesSubplot:>

In [50]:
np.linalg.norm(q[:,6,None]@q[:,2,None].T)

0.9999993

In [4]:
n_t=75
int(round(n_t * .1))

8

In [51]:
1/1.5

0.6666666666666666

In [116]:
plt.figure()
sns.heatmap(np.random.rand(4,4),cmap='Greys')

<AxesSubplot:>

In [118]:
u = np.random.rand(1,10)
v = np.random.rand(1,10)
plt.figure()
sns.heatmap(u.T@v,cmap='Greys')

<AxesSubplot:>

In [138]:
fig = plt.figure(figsize=(3.5,4))
plt.gcf().subplots_adjust(bottom=0.25)
gs = gridspec.GridSpec(2,2, figure=fig,width_ratios=[.25,1],height_ratios=[.25,1])

ax0 = fig.add_subplot(gs[0])
ax3 = fig.add_subplot(gs[3])
sns.heatmap(np.random.rand(4,4),ax=ax0,cbar=False)

u = np.random.rand(1,150)
v = np.random.rand(1,150)
sns.heatmap(gaussian_filter(u.T@v,sigma=2),ax=ax3,cbar=False)

for ax in [ax0,ax1,ax2,ax3]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(6)
    ax.tick_params(axis='both', which='major', pad=1)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.tick_params(axis='both', which='major', pad=1)
    ax.tick_params(bottom=False)
    ax.tick_params(left=False)
plt.savefig('/users/langdon/figures/circuitpaperfigures/schematic.png',dpi=1000, transparent=True)

In [141]:
inputs.shape

torch.Size([4050, 75, 6])

In [153]:
# Figure: input-output streams

In [185]:
inputs, labels, mask, conditions  = generate_trials(
                                            n_trials=25,
                                            alpha=float(0.2),
                                            tau=200,
                                            sigma_in=.01,
                                            baseline=0.2,
                                            n_coh=9)
fig = plt.figure(figsize=(4,1.2))

gs = gridspec.GridSpec(2,1, figure=fig,height_ratios=[1,.2],hspace=.5)

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
sns.heatmap(inputs[0,:,:].T,ax=ax0,cbar=False,yticklabels = ['Motion context','Color context','Left motion','Right motion', 'Left color', 'Right color'])
sns.heatmap(labels[0,:,:].T,ax=ax1,cbar=False, yticklabels = ['Right output', 'Left output'])



ax0.tick_params(axis='both', which='major', pad=1)
ax0.spines["top"].set_visible(False)
ax0.spines["right"].set_visible(False)
ax0.tick_params(axis='both', which='major', pad=1)
ax0.tick_params(bottom=False)
ax0.tick_params(left=False)
for label in ( ax0.get_yticklabels()+ ax0.get_yticklabels()):
    label.set_fontsize(4)
ax0.set_xticklabels([])
#ax0.set_yticklabels([])



for label in (ax1.get_xticklabels() + ax1.get_yticklabels()):
    label.set_fontsize(4)
ax1.tick_params(axis='both', which='major', pad=1)
ax1.spines["top"].set_visible(False)
ax1.spines["right"].set_visible(False)
#ax.set_xticklabels([])
#ax.set_yticklabels([])
ax1.tick_params(axis='both', which='major', pad=1)
ax1.tick_params(bottom=False)
ax1.tick_params(left=False)
plt.savefig('/users/langdon/figures/circuitpaperfigures/inputoutput.png',dpi=1000, transparent=True)

In [93]:
model_id = "04XOWLZl"

inputs, labels, mask, conditions  = generate_trials(
                                            n_trials=25,
                                            alpha=float(0.2),
                                            tau=200,
                                            sigma_in=.01,
                                            baseline=0.2,
                                            n_coh=4)

n = (Model() & {'model_id':model_id}).fetch1('n')
size = (Model() & {'model_id':model_id}).fetch1('connectivity')
rnn = RNNModule(connectivity=size, mask=mask, n=n)

rnn.recurrent_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_rec'))
rnn.input_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_in'))
rnn.output_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_out'))


In [38]:
df.groupby(['context','motion_coh','color_coh'])['x'].apply(np.mean).values[0].shape

(75, 150)

In [48]:
mean_x = df.groupby(['context','motion_coh','color_coh'])['x'].mean().values
mean_z = df.groupby(['context','motion_coh','color_coh'])['z'].apply(np.mean).values
labels = np.concatenate((np.stack(mean_x),np.stack(mean_z)), axis=2)

In [49]:
labels.shape

(128, 75, 152)

In [50]:
inputs.shape

torch.Size([3200, 75, 6])

In [52]:
len(conditions)

3200

In [85]:
df = pd.DataFrame(data=conditions)
df['x']=list(x)
df['z']=list(z)
df['inputs']=list(inputs.numpy())


mean_x = np.stack(df.groupby(['context','motion_coh','color_coh'])['x'].apply(np.mean).values)
mean_z = np.stack(df.groupby(['context','motion_coh','color_coh'])['z'].apply(np.mean).values)
labels = np.concatenate((mean_x,mean_z), axis=2)
#labels = np.concatenate((np.concatenate(mean_x,axis=0),np.concatenate(mean_z,axis=0)), axis=2)

mean_inputs = np.stack(df.groupby(['context','motion_coh','color_coh'])['inputs'].apply(np.mean).values)


In [90]:
plt.figure()
sns.heatmap(mean_inputs[10,:,:].T)

<AxesSubplot:>

In [89]:
plt.figure()
sns.heatmap(mean_z[10,:,:].T)

<AxesSubplot:>

In [69]:
np.concatenate(mean_x,axis=0).shape

(9600, 150)

In [83]:
mean_z.shape

(128,)

In [95]:
x,_,z = rnn.forward(inputs.to(device=device),training=False)
x = x.detach().cpu()
z = z.detach().cpu()
labels = torch.cat((x,z), dim=2)
df = pd.DataFrame(data=conditions)
df['labels']=list(labels.numpy())
df['inputs']=list(inputs.numpy())

mean_labels = df.groupby(['context','motion_coh','color_coh'])['labels'].apply(np.mean).values

TypeError: forward() got an unexpected keyword argument 'training'

In [99]:
n_trials = 25
inputs, labels, mask, conditions  = generate_trials(
                                            n_trials=n_trials,
                                            alpha=float(0.2),
                                            tau=200,
                                            sigma_in=.01,
                                            baseline=0.2,
                                            n_coh=4)

# Reconstruct model from model_id
model_id ="1aYzbM7K"
N = (Model_paper() & {'model_id':model_id}).fetch1('n')
rnn_net = RNNNet(
    module=RNNModule,
    module__n=N,
    module__connectivity='large',
    module__mask = mask,
    device=device,
)
rnn_net.initialize()
rnn_net.module_.recurrent_layer.weight.data = torch.tensor((Model_paper() & {'model_id':model_id}).fetch1('w_rec'), device= device)
rnn_net.module_.input_layer.weight.data = torch.tensor((Model_paper() & {'model_id':model_id}).fetch1('w_in'),device=device)
rnn_net.module_.output_layer.weight.data = torch.tensor((Model_paper() & {'model_id':model_id}).fetch1('w_out'),device=device)


In [137]:
x,_,z = rnn_net.forward(inputs.to(device=device),training=False)
x = x.detach().cpu()
z = z.detach().cpu()
labels = torch.cat((x,z), dim=2)
df = pd.DataFrame(data=conditions)
df['labels']=list(labels.numpy())
df['inputs']=list(inputs.numpy())
def group_mean(x):

    return np.mean(np.stack(x),axis=0)

mean_labels = torch.tensor(np.stack(df.groupby(['context','motion_coh','color_coh'])['labels'].apply(lambda x: group_mean(x)).reset_index()['labels'].values))
mean_inputs = torch.tensor(np.stack(df.groupby(['context','motion_coh','color_coh'])['inputs'].apply(lambda x: group_mean(x)).reset_index()['inputs'].values))

In [138]:
mean_labels.shape

torch.Size([128, 75, 152])

In [139]:
LCA_paper()

model_id unique model id,lca_id,alpha,sigma_rec,lr,weight_decay,patience,threshold,n_trials,batch_size,max_epochs,epochs,r2,r2_x,r2_xqt,r2_z,valid_loss,train_loss,valid_loss_history,train_loss_history,w_rec,w_in,w_out,q,a
1aYzbM7K,VLCkqPsp,0.20,0.15,0.0200000,0.00000,100,0.0001000,25,128,10000,162,0.7812,0.9400,0.7454,0.5979,0.21891,0.19609,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1aYzbM7K,wMw2NhMz,0.20,0.15,0.0200000,0.00000,100,0.0001000,25,128,10000,290,0.7395,0.9041,0.7181,0.6453,0.26055,0.27423,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1BcwkYD7,EpGbbAPl,0.20,0.00,0.0100000,0.00000,200,0.0000100,25,128,10000,282,0.7284,0.9538,0.8629,-1.1826,0.50907,0.27180,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1BcwkYD7,MTw6Y6bq,0.20,0.15,0.0100000,0.00000,200,0.0000100,25,128,10000,265,0.8435,0.9612,0.8104,0.7422,0.42260,0.15654,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1BcwkYD7,PiclN9nH,0.20,0.00,0.0100000,0.00000,200,0.0000100,25,128,10000,270,0.6997,0.9410,0.8308,-1.1892,0.53661,0.30027,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1BcwkYD7,V5qsl1FJ,0.20,0.00,0.0100000,0.00000,200,0.0000100,25,128,10000,313,0.7170,0.9508,0.8476,-1.1876,0.47040,0.28314,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
1BcwkYD7,Z6sSzXJn,0.20,0.00,0.0100000,0.00000,200,0.0000100,25,128,10000,275,0.7036,0.9491,0.8259,-1.1881,0.51433,0.29645,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=


In [122]:

def group_mean(x):
    return np.mean(np.stack(x),axis=0)

df.groupby(['context','motion_coh','color_coh'])['labels'].apply(lambda x: group_mean(x)).reset_index()

,context,motion_coh,color_coh,labels
0,color,-1.0,-1.000000,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,color,-1.0,-0.215443,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,color,-1.0,-0.046416,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,color,-1.0,-0.010000,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,color,-1.0,0.010000,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...
123,motion,1.0,-0.010000,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
124,motion,1.0,0.010000,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
125,motion,1.0,0.046416,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
126,motion,1.0,0.215443,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [116]:
df

,context,motion_coh,color_coh,correct_choice,labels,inputs
0,color,0.010000,-0.046416,-1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.18756938, 0.21589386, 0.22006351, 0.222384..."
1,motion,0.010000,0.046416,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.17260602, 0.23970601, 0.25003648, 0.222830..."
2,motion,0.215443,-0.010000,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.26418594, 0.16451976, 0.23228014, 0.223428..."
3,motion,0.046416,-0.215443,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.24511164, 0.17924628, 0.22814265, 0.192244..."
4,motion,-0.215443,-0.046416,-1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.2241921, 0.19603983, 0.16268082, 0.1942518..."
...,...,...,...,...,...,...
3195,motion,0.046416,-0.010000,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.16979079, 0.20973136, 0.16982177, 0.256700..."
3196,motion,-0.046416,1.000000,-1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.2548088, 0.18097349, 0.15883714, 0.1938228..."
3197,color,-0.010000,-0.215443,-1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.22951505, 0.19120848, 0.21558268, 0.200456..."
3198,motion,0.010000,0.215443,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.19086735, 0.15761945, 0.14485362, 0.189299..."


In [126]:
mean_labels.shape

(128, 75, 152)